In [1]:
pwd

'/Users/dblyon/modules/cpr/agotool'

In [2]:
cd app/python

/Users/dblyon/modules/cpr/agotool/app/python


In [3]:
import os, sys, socket

In [4]:
sys.version, sys.executable

('3.8.2 | packaged by conda-forge | (default, Apr 24 2020, 07:56:27) \n[Clang 9.0.1 ]',
 '/Users/dblyon/anaconda3/envs/agotool2/bin/python')

# Imports

In [5]:
import query, variables, userinput

In [7]:
%load_ext Cython

In [8]:
%reload_ext Cython

# Preload

In [9]:
import query, variables
from importlib import reload
reload(query)
reload(variables)
variables.DB_DOCKER = False
variables.DOCKER = False
# taxid_2_funcEnum_index_2_associations --> taxid_2_tuple_funcEnum_index_2_associations_counts
low_memory = True
### preload
pqo = query.PersistentQueryObject_STRING(low_memory)
static_preloaded_objects = pqo.get_static_preloaded_objects(low_memory)
### needed for scipy comparison, but otherwise deprecated
# ENSP_2_tuple_funcEnum_score_dict = query.get_proteinAN_2_tuple_funcEnum_score_dict(read_from_flat_files=True)

################################################################################
initializing PQO
getting taxid_2_proteome_count
getting taxid_2_proteins_dict
getting NCBI_taxonomy and TaxidSpecies_2_TaxidProteome_dict
getting lookup arrays
getting cond arrays
getting lineage dict
getting blacklisted terms
getting taxid_2_tuple_funcEnum_index_2_associations_counts
getting preloaded objects per analysis
finished with PQO init
go go GO and fly like the wind
################################################################################


In [10]:
year_arr, hierlevel_arr, entitytype_arr, functionalterm_arr, indices_arr, description_arr, category_arr, etype_2_minmax_funcEnum, function_enumeration_len, etype_cond_dict, etype_2_num_functions_dict, taxid_2_proteome_count, taxid_2_tuple_funcEnum_index_2_associations_counts, lineage_dict_enum, blacklisted_terms_bool_arr, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, kegg_taxid_2_acronym_dict, goslimtype_2_cond_dict = static_preloaded_objects

In [ ]:
# # DEBUG no background_counts for Ecoli
# taxid = 592
# funcEnum_index_2_associations = taxid_2_tuple_funcEnum_index_2_associations_counts[taxid]
# # funcEnum_index_positions_arr, counts_arr = funcEnum_index_2_associations
# # create_funcEnum_count_background_v3(funcEnum_count_background, funcEnum_index_positions_arr, counts_arr)

In [ ]:
# taxid = 511145 # 562
# taxid = 562
# taxid = 9606
# taxid = 83334
# taxid in taxid_2_tuple_funcEnum_index_2_associations_counts.keys()

In [ ]:
# ###############################################################################################
# ###############################################################################################
# ### example of CSR and CSC row and column slicing. DON'T DELETE
# from scipy.sparse import random
# m = random(5, 5, format='csr', density=0.25)
# m.data[:] = np.arange(1, m.data.shape[0]+1)
# csr = m.copy()
# print("### CSR matrix###")
# print("the data as an array")
# print(m.toarray())
# print()
# print("values only") # in row major order !!
# print(m.data)
# print()
# print("indices")
# print(m.indices) # index positions within each row that contains values
# print()
# print("indptr")
# print(m.indptr) # index positions of row start
# print()
# print("get row index and values (only if not empty)")
# for i in range(len(m.indptr[:-1])): # get row values
#     index_row_start = m.indptr[i]
#     index_row_stop = m.indptr[i+1]
#     if index_row_start == index_row_stop:
#         continue
#     print(i, m.data[index_row_start:index_row_stop])
# print()

# print("### now CSC ###")
# m = m.tocsc()
# csc = m.copy()
# print("the data as an array")
# print(m.toarray())
# print()
# print("values only") # now changed in column major order !!!
# print(m.data)
# print()
# print("indices")
# print(m.indices) # index positions within each column that contains values
# print()
# print("indptr")
# print(m.indptr) # index positions of column start and stop
# print()
# print("get column values")
# for i in range(len(m.indptr[:-1])): # get column values
#     index_row_start = m.indptr[i]
#     index_row_stop = m.indptr[i+1]
#     print(m.data[index_row_start:index_row_stop])
    
# print("get column values and column indices (only if not empty)") # column indices is correct since column data is returned and indexed
# for i in range(len(m.indptr[:-1])): # get column values
#     index_row_start = m.indptr[i]
#     index_row_stop = m.indptr[i+1]
#     if index_row_start == index_row_stop:
#         continue
#     print(i, m.data[index_row_start:index_row_stop])
    
    
# print("-"*50)    
# print("for each row that is not empty, get row index and column indices (not values)")
# for rowIndex in range(len(csr.indptr[:-1])): # get row values
#     index_row_start = csr.indptr[rowIndex]
#     index_row_stop = csr.indptr[rowIndex+1]
#     if index_row_start == index_row_stop:
#         continue
#     print(rowIndex, csr[rowIndex].indices)
# print()

# print("-"*50)    
# print("for each column that is not empty, get column index (funcEnum), values of column, and row indices within column (translate to ENSPs)")
# # use row_indices_per_column for tranlation from bg_val (score) to bg_rank
# # row_indices_per_column = csc.indices[index_row_start:index_row_stop]
# for colIndex in range(len(csc.indptr[:-1])): # get column values
#     index_row_start = csc.indptr[colIndex]
#     index_row_stop = csc.indptr[colIndex+1]
#     if index_row_start == index_row_stop:
#         continue
#     print(colIndex, csc.data[index_row_start:index_row_stop], csc.indices[index_row_start:index_row_stop])
#     print(csc[:, colIndex].todense())
#     print()
# print()

# # rowIndex = 1
# # csr[rowIndex].data, csr[rowIndex].indices, csr[rowIndex].indptr

# Functions

In [56]:
%%cython -f --compile-args=-DCYTHON_TRACE=1

import Cython
######################################
### profiling # Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
import line_profiler
directive_defaults = Cython.Compiler.Options.get_directive_defaults() ### from Cython.Compiler.Options import directive_defaults # deprecated
directive_defaults['linetrace'] = True
directive_defaults['binding'] = True
######################################
from functools import reduce
import math
import numpy as np
import pandas as pd
from cython cimport boundscheck, wraparound, cdivision, nonecheck
cimport cython
cimport numpy as np
ctypedef np.uint8_t uint8
from collections import defaultdict
from fisher import pvalue
from scipy import stats
import variables, query
import time



def run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=False, debug=False): #, KS_method="sparse_scipy", ENSP_2_tuple_funcEnum_score_dict=None, KS_etypes_FG_IDs=True):
    if not low_memory:
        ENSP_2_functionEnumArray_dict, year_arr, hierlevel_arr, entitytype_arr, functionalterm_arr, indices_arr, description_arr, category_arr, etype_2_minmax_funcEnum, function_enumeration_len, etype_cond_dict, etype_2_num_functions_dict, taxid_2_proteome_count, taxid_2_tuple_funcEnum_index_2_associations_counts, lineage_dict_enum, blacklisted_terms_bool_arr, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, kegg_taxid_2_acronym_dict, goslimtype_2_cond_dict = static_preloaded_objects
    else:  # missing: ENSP_2_functionEnumArray_dict
        year_arr, hierlevel_arr, entitytype_arr, functionalterm_arr, indices_arr, description_arr, category_arr, etype_2_minmax_funcEnum, function_enumeration_len, etype_cond_dict, etype_2_num_functions_dict, taxid_2_proteome_count, taxid_2_tuple_funcEnum_index_2_associations_counts, lineage_dict_enum, blacklisted_terms_bool_arr, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, kegg_taxid_2_acronym_dict, goslimtype_2_cond_dict = static_preloaded_objects
    foreground_ids_arr_of_string, background_ids_arr_of_string, funcEnum_count_foreground, funcEnum_count_background, p_values, p_values_corrected, cond_multitest, blacklisted_terms_bool_arr_temp, cond_terms_reduced_with_ontology, cond_filter, cond_PMIDs, effectSizes, over_under_int_arr, over_under_arr_of_string = preloaded_objects_per_analysis
    em = ui.enrichment_method
    foreground_n = ui.get_foreground_n()
    args_dict = ui.args_dict
    simplified_output = args_dict["simplified_output"]
    background_n = ui.get_background_n()
    protein_ans_fg = ui.get_foreground_an_set()
    taxid = args_dict["taxid"]
    filter_foreground_count_one = args_dict["filter_foreground_count_one"]
    p_value_cutoff = args_dict["p_value_cutoff"]
    cols_2_return_sort_order = variables.cols_2_return_sort_order[:]

    if ui.enrichment_method in {"abundance_correction", "compare_samples"}: # , "compare_groups"
        protein_ans_bg = ui.get_background_an_set()
    if low_memory:
        ENSP_2_functionEnumArray_dict = query.get_functionEnumArray_from_proteins(ui.get_all_individual_AN(), dict_2_array=True)
    ### add protein groups to ENSP_2_functionEnumArray_dict
    ENSP_2_functionEnumArray_dict = add_protein_groups_to_ENSP_2_functionEnumArray_dict(ENSP_2_functionEnumArray_dict, ui.get_all_unique_proteinGroups())

    count_all_terms(ENSP_2_functionEnumArray_dict, protein_ans_fg, funcEnum_count_foreground)

    ### count background
    if em == "genome":
        funcEnum_index_2_associations = taxid_2_tuple_funcEnum_index_2_associations_counts[taxid]
        funcEnum_index_positions_arr, counts_arr = funcEnum_index_2_associations
        create_funcEnum_count_background_v3(funcEnum_count_background, funcEnum_index_positions_arr, counts_arr)
    elif em == "abundance_correction":
        funcEnum_count_background = count_all_term_abundance_corrected(ui, ENSP_2_functionEnumArray_dict, funcEnum_count_background)
        background_n = foreground_n
    elif em == "compare_samples":
        count_all_terms(ENSP_2_functionEnumArray_dict, protein_ans_bg, funcEnum_count_background)
    else:
        args_dict["ERROR enrichment_method"] = "The 'enrichment_method' you've provided: '{}' doesn't exist / isn't implemented.".format(args_dict["enrichment_method"])
        return args_dict

    ## limit to given entity types
    cond_limit_2_entity_type = limit_to_entity_types(args_dict["limit_2_entity_type"], function_enumeration_len, etype_cond_dict, funcEnum_count_foreground)
    limit_to_go_subset(etype_cond_dict, args_dict["go_slim_subset"], goslimtype_2_cond_dict, funcEnum_count_foreground)
    o_or_u_or_both_encoding = args_dict["o_or_u_or_both_encoding"]

    ### calculate Fisher p-values and get bool array for multiple testing
    calc_pvalues(funcEnum_count_foreground, funcEnum_count_background, foreground_n, background_n, p_values, cond_multitest, effectSizes, over_under_int_arr, o_or_u_or_both_encoding)

    ######################################################################################################################################################
    ### Jensenlab Scores KS test
#     cond_KS_etypes = etype_cond_dict["cond_25"] | etype_cond_dict["cond_26"] | etype_cond_dict["cond_20"]

#     fg_scores_matrix, list_of_rowIndices_fg = slice_ScoresMatrix_for_given_ENSP(protein_ans_fg, ENSP_2_rowIndex_dict, CSC_ENSPencoding_2_FuncEnum)
#     fg_scores_matrix_data = fg_scores_matrix.data
#     fg_scores_matrix_indptr = fg_scores_matrix.indptr
#     if fg_scores_matrix_data.size > 0:
#         if em == "genome": # "genome" has 2 possible KS methods KolmogorovSmirnov_sparse_cy (if fg not a proper subset of bg but comparing to precomputed bg) and KolmogorovSmirnov_sparse_cy_genome.
#             if KS_method in {"cy", "sparse_scipy"}:
#                 bg_scores_matrix_data = None
#                 bg_scores_matrix_indptr = None
#                 try:
#                     funcEnum_2_scores_dict_bg = Taxid_2_FunctionEnum_2_Scores_dict[taxid] # taxid is an Integer
#                 except KeyError: # no text mining information for this taxon, try to translate to species level and try again. e.g. user provides 559292 (Saccharomyces cerevisiae S288C, UniProt Reference Proteome), but Jensenlab Textmining supports 4932 (Saccharomyces cerevisiae, rank species)
#                     funcEnum_2_scores_dict_bg = {} # TaxID check is already done in runserver.py and userinput.py
#                 if KS_method == "sparse_scipy":
#                     KolmogorovSmirnov_sparse_scipy(funcEnum_2_scores_dict_bg, foreground_n, background_n, fg_scores_matrix_data, fg_scores_matrix_indptr, bg_scores_matrix_data, bg_scores_matrix_indptr, p_values, cond_multitest, effectSizes, p_value_cutoff, funcEnum_count_foreground, funcEnum_count_background, over_under_int_arr, o_or_u_or_both_encoding, em, filter_foreground_count_one)
#                 elif KS_method == "cy":
#                     KolmogorovSmirnov_sparse_cy(funcEnum_2_scores_dict_bg, foreground_n, background_n, fg_scores_matrix_data, fg_scores_matrix_indptr, bg_scores_matrix_data, bg_scores_matrix_indptr, p_values, cond_multitest, effectSizes, p_value_cutoff, funcEnum_count_foreground, funcEnum_count_background, over_under_int_arr, o_or_u_or_both_encoding, em, filter_foreground_count_one)
#                 else:
#                     print("KS_method {} unknown".format(KS_method))
#                     return None

#             elif KS_method == "scipy":
#                 funcEnums_2_include_set = set(indices_arr[cond_KS_etypes & cond_limit_2_entity_type])
#                 funcEnum_2_scores_dict_fg = collect_scores_per_term_limit_2_inclusionTerms(protein_ans_fg, ENSP_2_tuple_funcEnum_score_dict, funcEnums_2_include_set, list_2_array=True)
#                 funcEnum_2_scores_dict_bg = Taxid_2_FunctionEnum_2_Scores_dict[taxid]
#                 if debug:
#                     return foreground_n, background_n, funcEnum_2_scores_dict_fg, funcEnum_2_scores_dict_bg, p_values, cond_multitest, effectSizes, p_value_cutoff, funcEnum_count_foreground, funcEnum_count_background, over_under_int_arr, o_or_u_or_both_encoding, True
#                 print("running KolmogorovSmirnov_scipy")
#                 KolmogorovSmirnov_scipy(foreground_n, background_n, funcEnum_2_scores_dict_fg, funcEnum_2_scores_dict_bg, p_values, cond_multitest, effectSizes, p_value_cutoff, funcEnum_count_foreground, funcEnum_count_background, over_under_int_arr, o_or_u_or_both_encoding, fill_zeros=True)
#             else:
#                 print("KS_method {} not implemented".format(KS_method))
#                 return None
#         elif em in {"compare_samples", "abundance_correction"}: # abundance_correction calculated the same way as compare_samples, background_n will differ from non-KS etypes
#             if KS_method in {"cy", "sparse_scipy"}:
#                 bg_scores_matrix, list_of_rowIndices_bg = slice_ScoresMatrix_for_given_ENSP(protein_ans_bg, ENSP_2_rowIndex_dict, CSC_ENSPencoding_2_FuncEnum)
#                 bg_scores_matrix_data = bg_scores_matrix.data
#                 bg_scores_matrix_indptr = bg_scores_matrix.indptr
#                 funcEnum_2_scores_dict_bg = None
#                 if em == "abundance_correction":
#                     background_n_temp = ui.background.shape[0]
#                 else:
#                     background_n_temp = background_n
#                 if debug:
#                     return funcEnum_2_scores_dict_bg, foreground_n, background_n_temp, fg_scores_matrix_data, fg_scores_matrix_indptr, bg_scores_matrix_data, bg_scores_matrix_indptr, p_values, cond_multitest, effectSizes, p_value_cutoff, funcEnum_count_foreground, funcEnum_count_background, over_under_int_arr, o_or_u_or_both_encoding, em, filter_foreground_count_one
#                 if KS_method == "cy":
#                     print("running KolmogorovSmirnov_sparse_cy")
#                     KolmogorovSmirnov_sparse_cy(funcEnum_2_scores_dict_bg, foreground_n, background_n_temp, fg_scores_matrix_data, fg_scores_matrix_indptr, bg_scores_matrix_data, bg_scores_matrix_indptr, p_values, cond_multitest, effectSizes, p_value_cutoff, funcEnum_count_foreground, funcEnum_count_background, over_under_int_arr, o_or_u_or_both_encoding, em, filter_foreground_count_one)
#                 elif KS_method == "sparse_scipy":
#                     print("running KolmogorovSmirnov_sparse_scipy")
#                     KolmogorovSmirnov_sparse_scipy(funcEnum_2_scores_dict_bg, foreground_n, background_n_temp, fg_scores_matrix_data, fg_scores_matrix_indptr, bg_scores_matrix_data, bg_scores_matrix_indptr, p_values, cond_multitest, effectSizes, p_value_cutoff, funcEnum_count_foreground, funcEnum_count_background, over_under_int_arr, o_or_u_or_both_encoding, em, filter_foreground_count_one)
#                 else:
#                     print("KS_method {} unknown".format(KS_method))
#                     return None
#             elif KS_method == "scipy":
#                 funcEnums_2_include_set = set(indices_arr[cond_KS_etypes & cond_limit_2_entity_type])
#                 funcEnum_2_scores_dict_fg = collect_scores_per_term_limit_2_inclusionTerms(protein_ans_fg, ENSP_2_tuple_funcEnum_score_dict, funcEnums_2_include_set, list_2_array=True)
#                 funcEnum_2_scores_dict_bg = collect_scores_per_term_limit_2_inclusionTerms(protein_ans_bg, ENSP_2_tuple_funcEnum_score_dict, funcEnums_2_include_set, list_2_array=True)
#                 print("running KolmogorovSmirnov_scipy")
#                 if debug:
#                     fill_zeros = True
#                     return foreground_n, background_n, funcEnum_2_scores_dict_fg, funcEnum_2_scores_dict_bg, p_values, cond_multitest, effectSizes, p_value_cutoff, funcEnum_count_foreground, funcEnum_count_background, over_under_int_arr, o_or_u_or_both_encoding, fill_zeros
#                 KolmogorovSmirnov_scipy(foreground_n, background_n, funcEnum_2_scores_dict_fg, funcEnum_2_scores_dict_bg, p_values, cond_multitest, effectSizes, p_value_cutoff, funcEnum_count_foreground, funcEnum_count_background, over_under_int_arr, o_or_u_or_both_encoding, fill_zeros=True)

    #### other methods e.g. abundance_correction, compare_samples are missing  #!!!
    ### don't delete "add_funcEnums_2_dict_CSC", not using due to speed and too many proteins in list --> but use for characterize_foreground
    # add_funcEnums_2_dict_CSC(protein_ans_fg, ENSP_2_functionEnumArray_dict, ENSP_2_rowIndex_dict, CSR_ENSPencoding_2_FuncEnum)

    ### "over/under"
    if o_or_u_or_both_encoding == 1: # overrepresented
        over_under_arr_of_string[over_under_int_arr == 1] = "o"
    elif o_or_u_or_both_encoding == 0: # both
        over_under_arr_of_string[over_under_int_arr == 1] = "o"
        over_under_arr_of_string[over_under_int_arr == 2] = "u"
    elif o_or_u_or_both_encoding == 2: # underrepresented
        over_under_arr_of_string[over_under_int_arr == 2] = "u"
    else: # check already done above
        return args_dict
    ### multiple testing per entity type, save results preformed p_values_corrected
    if args_dict["multiple_testing_per_etype"]:
        for etype_name, cond_etype in etype_cond_dict.items():
            num_total_tests = etype_2_num_functions_dict[etype_name]
            multiple_testing_per_entity_type(cond_etype, cond_multitest, p_values, p_values_corrected, indices_arr, num_total_tests)
    else:
        cond_all = np.ones(function_enumeration_len, dtype=bool)
        num_total_tests = cond_all.shape[0]
        multiple_testing_per_entity_type(cond_all, cond_multitest, p_values, p_values_corrected, indices_arr, num_total_tests)

    ### Filter stuff
#     if KS_etypes_FG_IDs:
#         add_funcEnums_2_dict_CSC(protein_ans_fg, ENSP_2_functionEnumArray_dict, ENSP_2_rowIndex_dict, CSR_ENSPencoding_2_FuncEnum)
    foreground_ids_arr_of_string, funcEnum_indices_for_IDs, cond_etypes_with_ontology_filtered, cond_etypes_rem_foreground_ids_filtered, cond_filter = filter_stuff(args_dict, protein_ans_fg, p_values_corrected, foreground_ids_arr_of_string, funcEnum_count_foreground, year_arr, p_values, indices_arr, ENSP_2_functionEnumArray_dict, cond_filter, etype_cond_dict, cond_PMIDs, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, over_under_int_arr)
    if debug:
        return foreground_ids_arr_of_string
    if em in {"compare_samples"}:
        background_ids_arr_of_string = map_funcEnum_2_ENSPs(protein_ans_bg, ENSP_2_functionEnumArray_dict, funcEnum_indices_for_IDs, background_ids_arr_of_string)

    ### filter etypes with ontologies --> cond_terms_reduced_with_ontology
    df_with_ontology = pd.DataFrame({"term_enum": indices_arr[cond_etypes_with_ontology_filtered].view(), "foreground_ids": foreground_ids_arr_of_string[cond_etypes_with_ontology_filtered].view(), "hierarchical_level": hierlevel_arr[cond_etypes_with_ontology_filtered].view(), "p_value": p_values[cond_etypes_with_ontology_filtered].view(), "foreground_count": funcEnum_count_foreground[cond_etypes_with_ontology_filtered].view(), "etype": entitytype_arr[cond_etypes_with_ontology_filtered].view()})
    if args_dict["filter_parents"]: # only for etypes with ontology, but since foreground IDs needed get them for all
        filter_parents_if_same_foreground(blacklisted_terms_bool_arr_temp, cond_terms_reduced_with_ontology, lineage_dict_enum, df_with_ontology) # modifies cond_terms_reduced_with_ontology inplace
    else: # since no filtering done use all etypes with ontology
        cond_terms_reduced_with_ontology = cond_filter & cond_etypes_with_ontology
    ### concatenate filtered results
    cond_2_return = cond_PMIDs | cond_terms_reduced_with_ontology | cond_etypes_rem_foreground_ids_filtered

    if simplified_output:
        df_2_return = pd.DataFrame({"term": functionalterm_arr[cond_2_return].view(),
                                "hierarchical_level": hierlevel_arr[cond_2_return].view(),
                                "p_value": p_values[cond_2_return].view(),
                                "FDR": p_values_corrected[cond_2_return].view(),
                                "category": category_arr[cond_2_return].view(),
                                "etype": entitytype_arr[cond_2_return].view(),
                                "description": description_arr[cond_2_return].view(),
                                "year": year_arr[cond_2_return].view(),
                                "FG_IDs": foreground_ids_arr_of_string[cond_2_return].view(),
                                "FG_count": funcEnum_count_foreground[cond_2_return].view(),
                                "BG_count": funcEnum_count_background[cond_2_return].view()})
        return df_2_return[['term', 'hierarchical_level', 'p_value', 'FDR', 'category', 'etype', 'description', 'FG_count', 'BG_count', 'FG_IDs', 'year']]

    df_2_return = pd.DataFrame({"term": functionalterm_arr[cond_2_return].view(),
                            "hierarchical_level": hierlevel_arr[cond_2_return].view(),
                            "p_value": p_values[cond_2_return].view(),
                            "FDR": p_values_corrected[cond_2_return].view(),
                            "category": category_arr[cond_2_return].view(),
                            "etype": entitytype_arr[cond_2_return].view(),
                            "description": description_arr[cond_2_return].view(),
                            "year": year_arr[cond_2_return].view(),
#                             "ratio_in_FG": ratio_in_foreground[cond_2_return].view(),
#                             "ratio_in_BG": ratio_in_background[cond_2_return].view(),
                            "FG_IDs": foreground_ids_arr_of_string[cond_2_return].view(),
                            "FG_count": funcEnum_count_foreground[cond_2_return].view(),
                            "BG_count": funcEnum_count_background[cond_2_return].view(),
                            "effectSize": effectSizes[cond_2_return].view(),
                            "over_under": over_under_arr_of_string[cond_2_return].view(),
                            "funcEnum": indices_arr[cond_2_return].view()})

    if em in {"compare_samples"}: # , "compare_groups"
        df_2_return["BG_IDs"] = background_ids_arr_of_string[cond_2_return].view()
    else:
        cols_2_return_sort_order.remove("BG_IDs")
    df_2_return = s_value(df_2_return)
    df_2_return["s_value_abs"] = df_2_return["s_value"].apply(lambda x: abs(x))
    df_2_return = df_2_return.sort_values(["etype", "s_value_abs", "hierarchical_level", "year"], ascending=[False, False, False, False])
    df_2_return["rank"] = df_2_return.groupby("etype")["s_value_abs"].rank(ascending=False, method="first").fillna(value=df_2_return.shape[0]).astype(int)
    if debug:
            return protein_ans_bg, ENSP_2_functionEnumArray_dict, funcEnum_indices_for_IDs, background_ids_arr_of_string, df_2_return
    df_2_return = ui.translate_primary_back_to_secondary(df_2_return)
    df_2_return["FG_n"] = foreground_n
    df_2_return["BG_n"] = background_n
#     if em == "abundance_correction":
#         cond_KS_etypes_temp = df_2_return["etype"].isin(variables.entity_types_with_scores)
#         df_2_return.loc[cond_KS_etypes_temp, "BG_n"] = ui.background.shape[0]
    # #!!! DEBUG uncomment for production #     df_2_return.loc[df_2_return["etype"].isin([-20, -25, -26]), ["ratio_in_FG", "ratio_in_BG", "FG_count", "BG_count"]] = np.nan

    ### calc ratio in foreground, count foreground / len(protein_ans)
    df_2_return["ratio_in_FG"] = df_2_return["FG_count"] / df_2_return["FG_n"] # ratio_in_foreground = funcEnum_count_foreground / foreground_n
    df_2_return["ratio_in_BG"] = df_2_return["BG_count"] / df_2_return["BG_n"] # ratio_in_background = funcEnum_count_background / background_n
    try: 
        STRING_beta = args_dict["STRING_beta"]
    except KeyError:
        STRING_beta = False
    if STRING_beta:
        df_2_return = df_2_return.rename(columns={"BG_count": 'background_count', "FG_count": 'foreground_count', "FG_IDs": 'foreground_ids'})
        return df_2_return[variables.cols_sort_order_genome_STRING_beta + list(set(df_2_return.columns.tolist()) - set(variables.cols_sort_order_genome_STRING_beta))]
    return df_2_return[cols_2_return_sort_order]

@boundscheck(False)
@wraparound(False)
cdef set_fg_counts(unsigned int [::1] fg_scores_matrix_data, int [::1] fg_scores_matrix_indptr, unsigned int[::1] funcEnum_count_foreground, filter_foreground_count_one):
    cdef:
        unsigned int len_fg_scores_matrix_indptr
        unsigned int funcEnum, num_fg_vals

    len_fg_scores_matrix_indptr = fg_scores_matrix_indptr.shape[0]
    for funcEnum in range(len_fg_scores_matrix_indptr - 1):
        index_col_start_fg = fg_scores_matrix_indptr[funcEnum]
        index_col_stop_fg = fg_scores_matrix_indptr[funcEnum + 1]
        if index_col_start_fg == index_col_stop_fg:
            continue # column is empty
        elif filter_foreground_count_one and (index_col_stop_fg - index_col_start_fg) == 1:
            continue
        else:
            fg_values = fg_scores_matrix_data[index_col_start_fg:index_col_stop_fg]
        num_fg_vals = fg_values.shape[0]
        funcEnum_count_foreground[funcEnum] = num_fg_vals

@boundscheck(False)
@wraparound(False)
cdef int collect_scores_per_term_limit_2_inclusionTerms_arr_2(protein_AN_set, ENSP_2_tuple_funcEnum_score_dict, funcEnums_2_include_set, unsigned int[:, ::1] funcEnumContiguousIndex_2_Scores_arr, unsigned int[::1] funcEnum_2_funcEnumIndex_arr):
    """
    # unsigned int[:, ::1] arr
    for a given protein: a functional term should only have a single score (not multiple as previously)
    ENSP_2_tuple_funcEnum_score_dict['3702.AT1G01010.1']
    (array([ 211,  252,  253], dtype=uint32),
     array([420000, 4166357, 4195121], dtype=uint32))
    funcEnum_2_scores_array: 2D array of Zeros unless filled with Jensenlab-Score, 
        shape=(len_funcEnums_2_include_set, len_protein_AN_list)
        row number indirectly codes for funcEnum, column codes for AN enumeration (which does not need to be preserved, since sorted afterwards)
    some funcEnum rows will stay empty since entity_types_with_scores = {-20, -25, -26}  # GO-CC, BTO, DOID
    are not contiguous numbers.
    """
    cdef:
        unsigned int index_protein
        unsigned int index_funcEnum
        unsigned int funcEnum_contiguous_index
        unsigned int len_funcEnum_arr
        unsigned int funcEnum
        unsigned int score
        const unsigned int[::1] funcEnum_arr
        const unsigned int[::1] score_arr
        unsigned int max_funcEnums_2_include_set = max(funcEnums_2_include_set)

    for index_protein, protein_AN in enumerate(protein_AN_set): # row-index
        try:
            funcEnum_score = ENSP_2_tuple_funcEnum_score_dict[protein_AN]
        except KeyError:
            continue
        funcEnum_arr = funcEnum_score[0]
        score_arr = funcEnum_score[1]
        len_funcEnum_arr = funcEnum_arr.shape[0]
        for index_funcEnum in range(len_funcEnum_arr): # not col-index
            funcEnum = funcEnum_arr[index_funcEnum]
            if funcEnum <= max_funcEnums_2_include_set: # remove for speed-up, [filter later #!!! ToDo]
                score = score_arr[index_funcEnum]
                funcEnum_contiguous_index = funcEnum_2_funcEnumIndex_arr[funcEnum] # col-index, since enum
                funcEnumContiguousIndex_2_Scores_arr[funcEnum_contiguous_index, index_protein] = score
    return 0

@boundscheck(False)
@wraparound(False)
cdef int create_funcEnum_count_background_v3(unsigned int[::1] funcEnum_count_background,
                                         const unsigned int[::1] funcEnum_index_arr, # uint32
                                         const unsigned int[::1] count_arr): # uint32
    cdef:
        int i, N = funcEnum_index_arr.shape[0]
        unsigned int index_
        unsigned short count

    for i in range(N):
        index_ = funcEnum_index_arr[i]
        count = count_arr[i]
        funcEnum_count_background[index_] = count
    return 0

def count_all_term_abundance_corrected(ui, ENSP_2_functionEnumArray_dict, funcEnum_count):
    funcEnum_count_float = np.zeros(funcEnum_count.shape[0], dtype=np.dtype("float64"))
    for proteinGroup_list, correction_factor in ui.iter_bins():
        for proteinGroup in proteinGroup_list:
            try:
                funcEnum_associations = ENSP_2_functionEnumArray_dict[proteinGroup]
            except KeyError: # no functional annotation for proteins
                continue
            if proteinGroup in {'A0A2L0PMR4_VITS1','A0A2L0PNY0_VITS1', 'A0A2L0PNZ2_VITS1','A0A2L0PPD5_VITS1', 'Q8L311_VITS1'}:
                print(proteinGroup, correction_factor, funcEnum_associations)
            count_terms_cy_abundance_corrected(correction_factor, funcEnum_associations, funcEnum_count_float)
    funcEnum_count = np.around(funcEnum_count_float).astype(dtype=np.dtype("uint32"))
    return funcEnum_count

@boundscheck(False)
@wraparound(False)
cdef int count_terms_cy_abundance_corrected(double correction_factor,
                                        unsigned int[::1] funcEnum_associations,
                                        double[::1] funcEnum_count_float):
    cdef int N, i, k
    N = funcEnum_associations.shape[0]
    for i in range(N):
        k = funcEnum_associations[i]
        funcEnum_count_float[k] += correction_factor
    return 0

def count_all_terms(ENSP_2_functionEnumArray_dict, protein_ans, funcEnum_count):
    for ENSP in (ENSP for ENSP in protein_ans if ENSP in ENSP_2_functionEnumArray_dict):
        funcEnumAssociations = ENSP_2_functionEnumArray_dict[ENSP]
        count_terms_cy(funcEnumAssociations, funcEnum_count)

@boundscheck(False)
@wraparound(False)
cdef int count_terms_cy(unsigned int[::1] funcEnum_associations,
                    unsigned int[::1] funcEnum_count):
    """
    without returning 'funcEnum_count' the function does inplace change of 'funcEnum_count'
    :param funcEnum_associations: np.array (of variable length, with functional associations 
    as enumerations (instead of strings), 
    uint32, i.e. which functional associations are given for provided user input proteins)
    :param funcEnum_count: np.array (shape of array from 0 to max enumeration of functional-terms, 
    uint32, each position codes for 
    a specific functional term, the value is a count for the given user input)
    :return: None
    """
    cdef int N, i, k
    N = funcEnum_associations.shape[0]
    for i in range(N):
        k = funcEnum_associations[i]
        funcEnum_count[k] += 1
    return 0

def collect_scores_per_term_characterize_foreground(protein_AN_list, ENSP_2_tuple_funcEnum_score_dict, funcEnums_2_include_set, score_cutoff=3):
    funcEnum_2_scores_dict = defaultdict(lambda: [])
    for protein_AN in protein_AN_list:
        funcEnum_already_counted = set()
        try:
            funcEnum_score = ENSP_2_tuple_funcEnum_score_dict[protein_AN]
        except KeyError:
            continue
        funcEnum_arr, score_arr = funcEnum_score
        len_funcEnum_arr = len(funcEnum_arr)
        for index_ in range(len_funcEnum_arr):
            funcEnum = funcEnum_arr[index_]
            if funcEnum in funcEnums_2_include_set:
                score = score_arr[index_]
                if score >= score_cutoff:
                    if funcEnum not in funcEnum_already_counted:
                        # in order to count a function only once per protein
                        funcEnum_2_scores_dict[funcEnum].append(score)
                        funcEnum_already_counted.update(set([funcEnum]))
    return funcEnum_2_scores_dict

def collect_scores_per_term(protein_AN_list, ENSP_2_tuple_funcEnum_score_dict, list_2_array=False):
    """
    ENSP_2_tuple_funcEnum_score_dict['3702.AT1G01010.1']
    (array([ 211,  252,  253], dtype=uint32),
     array([4200000, 4166357, 4195121], dtype=uint32))
    funcEnum_2_scores_dict: key: functionEnumeration, val: list of scores
    """
    funcEnum_2_scores_dict = defaultdict(lambda: [])
    for protein_AN in protein_AN_list:
        try:
            funcEnum_score = ENSP_2_tuple_funcEnum_score_dict[protein_AN]
        except KeyError:
            continue
        funcEnum_arr, score_arr = funcEnum_score
        len_funcEnum_arr = len(funcEnum_arr)
        for index_ in range(len_funcEnum_arr):
            score = score_arr[index_]
            funcEnum_2_scores_dict[funcEnum_arr[index_]].append(score)
    if list_2_array:
        return {funcEnum: np.asarray(scores, dtype=np.dtype(variables.dtype_TM_score)) for funcEnum, scores in funcEnum_2_scores_dict.items()} # float64 --> uint32
    # since concatenating np.arrays later on (for filling with zeros) produces 64 bit array anyway
    else:
        return funcEnum_2_scores_dict

def collect_scores_per_term_limit_2_inclusionTerms(protein_AN_list, ENSP_2_tuple_funcEnum_score_dict, funcEnums_2_include_set, list_2_array=False):
    """
    for a given protein: a functional term should only have a single score (not multiple as previously)
    ENSP_2_tuple_funcEnum_score_dict['3702.AT1G01010.1']
    (array([ 211,  252,  253], dtype=uint32),
     array([420000, 4166357, 4195121], dtype=uint32))
    funcEnum_2_scores_dict: key: functionEnumeration, val: list of Integer scores ( )
    """
    len_protein_AN_list = len(protein_AN_list)
    funcEnum_2_scores_dict = defaultdict(lambda: [0]*len_protein_AN_list)
    for index_protein, protein_AN in enumerate(protein_AN_list):
        try:
            funcEnum_score = ENSP_2_tuple_funcEnum_score_dict[protein_AN]
        except KeyError:
            continue
        funcEnum_arr, score_arr = funcEnum_score
        len_funcEnum_arr = len(funcEnum_arr)
        for index_ in range(len_funcEnum_arr):
            funcEnum = funcEnum_arr[index_]
            if funcEnum in funcEnums_2_include_set:
                score = score_arr[index_]
                funcEnum_2_scores_dict[funcEnum][index_protein] = score # funcEnum_2_scores_dict[funcEnum].append(score)
    if list_2_array:
        return {funcEnum: np.asarray(scores, dtype=np.dtype(variables.dtype_TM_score)) for funcEnum, scores in funcEnum_2_scores_dict.items()}
    # since concatenating np.arrays later on (for filling with zeros) produces 64 bit array anyway
    else:
        return funcEnum_2_scores_dict

def collect_scores_per_term_abundance_corrected(ui, ENSP_2_tuple_funcEnum_score_dict, funcEnums_2_include_set, list_2_array=False):
    funcEnum_2_scores_dict = defaultdict(lambda: [])
    for proteinGroup_list, correction_factor in ui.iter_bins():
        for proteinGroup in proteinGroup_list:
            try:
                funcEnum_score = ENSP_2_tuple_funcEnum_score_dict[proteinGroup]
            except KeyError:
                continue
            funcEnum_arr, score_arr = funcEnum_score
            len_funcEnum_arr = len(funcEnum_arr)
            for index_ in range(len_funcEnum_arr):
                funcEnum = funcEnum_arr[index_]
                if funcEnum in funcEnums_2_include_set:
                    score = score_arr[index_]
                    funcEnum_2_scores_dict[funcEnum].append(score*correction_factor)
    if list_2_array:
        return {funcEnum: np.asarray(scores, dtype=np.dtype(variables.dtype_TM_score)) for funcEnum, scores in funcEnum_2_scores_dict.items()}
        # since concatenating np.arrays later on (for filling with zeros) produces 64 bit array anyway
    else:
        return funcEnum_2_scores_dict

@boundscheck(False)
@wraparound(False)
cdef int calc_pvalues_orig(unsigned int[::1] funcEnum_count_foreground,
                  unsigned int[::1] funcEnum_count_background,
                  unsigned int foreground_n,
                  unsigned int background_n,
                  double[::1] p_values,
                  cond_multitest,
                  double[::1] effectSizes,
                  unsigned int[::1] over_under_int_arr,
                  unsigned int o_or_u_or_both_encoding):
    cdef:
        int index_, foreground_count, background_count, a, b, c, d
        int len_functions = funcEnum_count_foreground.shape[0]
        dict fisher_dict = {}
        double p_val_uncorrected
        double odds_ratio

    for index_ in range(len_functions):
        foreground_count = funcEnum_count_foreground[index_]
        if foreground_count == 0: # continue and leave p-value set to 1, no multiple testing
            continue
        if foreground_count == 1: # leave p-value set to 1, BUT DO multiple testing
            cond_multitest[index_] = True
            over_under_int_arr[index_] = 3 # meaningless encoding in order not to filter out things later if p_value_cutoff == 1
        else: # calculate p-value and do multiple testing
            cond_multitest[index_] = True
            background_count = funcEnum_count_background[index_]
            a = foreground_count # number of proteins associated with given GO-term
            b = foreground_n - foreground_count # number of proteins not associated with GO-term
            c = background_count
            d = background_n - background_count
            p_val_uncorrected = fisher_dict.get((a, b, c, d), -1)
            if p_val_uncorrected == -1:
                if o_or_u_or_both_encoding == 1: # overrepresented
                    p_val_uncorrected = pvalue(a, b, c, d).right_tail
                    over_under_int_arr[index_] = 1
                elif o_or_u_or_both_encoding == 0: # both
                    p_val_uncorrected = pvalue(a, b, c, d).two_tail
                    try:
                        is_greater = (a / (a + b)) > (c / (c + d))
                        if is_greater:
                            is_greater = 1
                        else:
                            is_greater = 2
                    except ZeroDivisionError:
                        is_greater = 0 # np.nan
                    over_under_int_arr[index_] = is_greater
                elif o_or_u_or_both_encoding == 2: # underrepresented
                    p_val_uncorrected = pvalue(a, b, c, d).left_tail
                    over_under_int_arr[index_] = 2
                else:
                    p_val_uncorrected = 1
                    over_under_int_arr[index_] = 3
                fisher_dict[(a, b, c, d)] = p_val_uncorrected
            else: # write over_under but don't calc pvalue
                if o_or_u_or_both_encoding == 1: # overrepresented
                    over_under_int_arr[index_] = 1
                elif o_or_u_or_both_encoding == 0: # both
                    try:
                        is_greater = (a / (a + b)) > (c / (c + d))
                    except ZeroDivisionError:
                        is_greater = np.nan
                    over_under_int_arr[index_] = is_greater
                elif o_or_u_or_both_encoding == 2: # underrepresented
                    over_under_int_arr[index_] = 2
                else:
                    over_under_int_arr[index_] = 3 # which caser is this supposed to be?
            p_values[index_] = p_val_uncorrected
            try:
                # https://stats.stackexchange.com/questions/22508/effect-size-for-fishers-exact-test
                # odds_ratio = (a * d) / (b * c) # true odds ratio
                # odds_ratio = (d / (c + d)) - (a / (a + b)) # difference in proportions
                odds_ratio = (a / (a + b)) - (c / (c + d)) # difference in proportions DBL
                # odds_ratio = (a / (a + b)) / (c / (c + d)) # from old agotool, ratio of percent in fg to percent in bg
            except ZeroDivisionError:
                odds_ratio = np.nan
            effectSizes[index_] = odds_ratio
    return 0

@boundscheck(False)
@wraparound(False)
cdef calc_pvalues(unsigned int[::1] funcEnum_count_foreground,
                  unsigned int[::1] funcEnum_count_background,
                  unsigned int foreground_n,
                  unsigned int background_n,
                  double[::1] p_values,
                  cond_multitest,
                  double[::1] effectSizes,
                  unsigned int[::1] over_under_int_arr,
                  unsigned int o_or_u_or_both_encoding):
    cdef:
        int index_, foreground_count, background_count, a, b, c, d
        int len_functions = funcEnum_count_foreground.shape[0]
        dict fisher_dict = {}
        double p_val_uncorrected
        double odds_ratio

    for index_ in range(len_functions):
        foreground_count = funcEnum_count_foreground[index_]
        if foreground_count > 0:
            cond_multitest[index_] = True
            over_under_int_arr[index_] = 3 # meaningless encoding in order not to filter out things later if p_value_cutoff == 1
            if foreground_count == 1: # leave p-value set to 1, BUT DO multiple testing
                continue
            background_count = funcEnum_count_background[index_]
            a = foreground_count # number of proteins associated with given GO-term
            b = foreground_n - foreground_count # number of proteins not associated with GO-term
            c = background_count
            d = background_n - background_count
            p_val_uncorrected = fisher_dict.get((a, b, c, d), -1)
            if p_val_uncorrected == -1:
                if o_or_u_or_both_encoding == 1: # overrepresented
                    p_val_uncorrected = pvalue(a, b, c, d).right_tail
                    over_under_int_arr[index_] = 1
                elif o_or_u_or_both_encoding == 0: # both
                    p_val_uncorrected = pvalue(a, b, c, d).two_tail
                    try:
                        is_greater = (a / (a + b)) > (c / (c + d))
                        if is_greater:
                            is_greater = 1
                        else:
                            is_greater = 2
                    except ZeroDivisionError:
                        is_greater = 0 # np.nan
                    over_under_int_arr[index_] = is_greater
                elif o_or_u_or_both_encoding == 2: # underrepresented
                    p_val_uncorrected = pvalue(a, b, c, d).left_tail
                    over_under_int_arr[index_] = 2
                else:
                    p_val_uncorrected = 1
                    over_under_int_arr[index_] = 3
                fisher_dict[(a, b, c, d)] = p_val_uncorrected
            else: # write over_under but don't calc pvalue
                if o_or_u_or_both_encoding == 1: # overrepresented
                    over_under_int_arr[index_] = 1
                elif o_or_u_or_both_encoding == 0: # both
                    try:
                        is_greater = (a / (a + b)) > (c / (c + d))
                    except ZeroDivisionError:
                        is_greater = np.nan
                    over_under_int_arr[index_] = is_greater
                elif o_or_u_or_both_encoding == 2: # underrepresented
                    over_under_int_arr[index_] = 2
                else:
                    over_under_int_arr[index_] = 3 # which caser is this supposed to be?
            p_values[index_] = p_val_uncorrected
            try:
                # https://stats.stackexchange.com/questions/22508/effect-size-for-fishers-exact-test
                # odds_ratio = (a * d) / (b * c) # true odds ratio
                # odds_ratio = (d / (c + d)) - (a / (a + b)) # difference in proportions
                odds_ratio = (a / (a + b)) - (c / (c + d)) # difference in proportions DBL
                # odds_ratio = (a / (a + b)) / (c / (c + d)) # from old agotool, ratio of percent in fg to percent in bg
            except ZeroDivisionError:
                odds_ratio = np.nan
            effectSizes[index_] = odds_ratio
    return 0

@boundscheck(False)
@wraparound(False)
cpdef int calc_pvalues_compare_groups(unsigned int[::1] funcEnum_count_foreground,
                  unsigned int[::1] funcEnum_count_background,
                  unsigned int[::1] funcEnum_count_foreground_redundant,
                  unsigned int[::1] funcEnum_count_background_redundant,
                  unsigned int foreground_replicates,
                  unsigned int background_replicates,
                  double[::1] p_values,
                  cond_multitest,
                  double[::1] effectSizes,
                  unsigned int[::1] over_under_int_arr,
                  unsigned int o_or_u_or_both_encoding):
    cdef:
        int index_, foreground_count, background_count, a, b, c, d, foreground_n, background_n
        int len_functions = funcEnum_count_foreground_redundant.shape[0]
        dict fisher_dict = {}
        double p_val_uncorrected
        double odds_ratio

    for index_ in range(len_functions):
        foreground_count = funcEnum_count_foreground_redundant[index_]
        if foreground_count == 0:
            # continue and leave p-value set to 1, no multiple testing
            continue
        elif foreground_count == 1:
            # leave p-value set to 1, BUT DO multiple testing
            cond_multitest[index_] = True
            over_under_int_arr[index_] = 3 # meaningless encoding in order not to filter out things later if p_value_cutoff == 1
        else:
            # calculate p-value and do multiple testing
            background_count = funcEnum_count_background_redundant[index_]
            cond_multitest[index_] = True
            a = foreground_count # number of proteins associated with given GO-term
            foreground_n = funcEnum_count_foreground[index_] * foreground_replicates
            b = foreground_n - foreground_count # number of proteins not associated with GO-term
            background_n = funcEnum_count_background[index_] * background_replicates
            c = background_count
            if background_count == 0:
                cond_multitest[index_] = True
                over_under_int_arr[index_] = 3
                continue
            d = background_n - background_count
            p_val_uncorrected = fisher_dict.get((a, b, c, d), -1)
            if p_val_uncorrected == -1:
                if o_or_u_or_both_encoding == 1: # overrepresented
                    p_val_uncorrected = pvalue(a, b, c, d).right_tail
                    over_under_int_arr[index_] = 1
                elif o_or_u_or_both_encoding == 0: # both
                    p_val_uncorrected = pvalue(a, b, c, d).two_tail
                    try:
                        is_greater = (a / (a + b)) > (c / (c + d))
                        if is_greater:
                            is_greater = 1
                        else:
                            is_greater = 2
                    except ZeroDivisionError:
                        is_greater = 0 # np.nan
                    over_under_int_arr[index_] = is_greater
                elif o_or_u_or_both_encoding == 2: # underrepresented
                    p_val_uncorrected = pvalue(a, b, c, d).left_tail
                    over_under_int_arr[index_] = 2
                else:
                    p_val_uncorrected = 1
                    over_under_int_arr[index_] = 3
                fisher_dict[(a, b, c, d)] = p_val_uncorrected
            else: # write over_under but don't calc pvalue
                if o_or_u_or_both_encoding == 1: # overrepresented
                    over_under_int_arr[index_] = 1
                elif o_or_u_or_both_encoding == 0: # both
                    try:
                        is_greater = (a / (a + b)) > (c / (c + d))
                    except ZeroDivisionError:
                        is_greater = np.nan
                    over_under_int_arr[index_] = is_greater
                elif o_or_u_or_both_encoding == 2: # underrepresented
                    over_under_int_arr[index_] = 2
                else:
                    over_under_int_arr[index_] = 3 # which case is this supposed to be?
            p_values[index_] = p_val_uncorrected
            try:
                odds_ratio = (a / (a + b)) - (c / (c + d)) # difference in proportions DBL
            except ZeroDivisionError:
                odds_ratio = np.nan
            effectSizes[index_] = odds_ratio
    return 0

@boundscheck(False)
@wraparound(False)
@cdivision(True)
cdef BenjaminiHochberg_cy(double[::1] p_values,
                         unsigned int num_total_tests,
                         double[::1] p_values_corrected,
                         unsigned int[::1] indices_2_BH):
    """
    #!!! cpdef instead of cdef for scores debugging/profiling
    ein index array mit absoluten positionen, pvals absolut und pvalscorr absolut
    p_values_2_BH, p_values_2_BH.shape[0], p_values_corrected_2_BH, indices_of_p_values_2_BH)
    :param p_values: unsorted array of float
    :param num_total_tests: Integer (number of all possible tests within etype/category, regardless of input)
    :param p_values_corrected: array of float (1.0 by default), shape is full function_enumeration_len NOT p_values    
    :param indices_2_BH: indices of superset, shape of array reduced to p_values_2_BH
    iterate over p_values in p_values_2_BH_sort_order
    set p_value_corrected at positions from indices_2_BH[p_values_2_BH_sort_order]
    """
    cdef:
        double prev_bh_value = 0.0
        double p_value, bh_value
        unsigned int index_2_BH, i
        unsigned int enum_counter = 1
        unsigned int N = indices_2_BH.shape[0]

    for i in range(N):
        index_2_BH = indices_2_BH[i]
        p_value = p_values[index_2_BH]
        bh_value = p_value * num_total_tests / enum_counter
        # Sometimes this correction can give values greater than 1,
        # so we set those values at 1
        bh_value = min(bh_value, 1)
        # To preserve monotonicity in the values, we take the
        # maximum of the previous value or this one, so that we
        # don't yield a value less than the previous.
        bh_value = max(bh_value, prev_bh_value)
        prev_bh_value = bh_value
        p_values_corrected[index_2_BH] = bh_value
        enum_counter += 1

def map_funcEnum_2_ENSPs(protein_ans_list, ENSP_2_functionEnumArray_dict, funcEnum_indices, foreground_ids_arr_of_string):
    """
    previously named get_foreground_IDs_arr now map_funcEnum_2_ENSPs
    for given protein_ans produce concatenate strings of ENSP associations
    :param protein_ans_list: List of String (or array), user provided ENSPs
    :param ENSP_2_functionEnumArray_dict: key: String, val: array of uint32, all ENSP to function enum associations
    :param funcEnum_indices: array of uint32, relevant func enums after filtering
    :param foreground_ids_arr_of_string: list of empty string, len of function_enumeration_len, list instead of array since len of longest string unknown and would take lots of memory
    :return: List of String of len function_enumeration_len with comma sep ENSPs at index positions coding for func enum
    """
    funcEnum_2_ENSPs_dict = {index_: [] for index_ in funcEnum_indices}
    for ENSP in protein_ans_list:
        try:
            functionEnumArray = ENSP_2_functionEnumArray_dict[ENSP]
        except KeyError: # happens since some ENSPs are without functional associations (or if single association in genome it is filtered out)
            continue
        for funcEnum in functionEnumArray:
            if funcEnum in funcEnum_2_ENSPs_dict:
                funcEnum_2_ENSPs_dict[funcEnum].append(ENSP)

    for funcEnum, ENSPs in funcEnum_2_ENSPs_dict.items():
        foreground_ids_arr_of_string[funcEnum] = ";".join(sorted(set(ENSPs))) # needs to be sorted otherwise grouping incorrect later on
    return foreground_ids_arr_of_string

def get_preloaded_objects_for_single_analysis(blacklisted_terms_bool_arr, function_enumeration_len=6834675):
    """
    funcEnum_count_foreground, funcEnum_count_background, p_values, p_values_corrected, cond_multitest, blacklisted_terms_bool_arr_temp, cond_terms_reduced_with_ontology, foreground_ids_arr_of_string, cond_filter, cond_PMIDs
    """
    funcEnum_count_foreground = np.zeros(shape=function_enumeration_len, dtype=np.dtype("uint32"))
    foreground_ids_arr_of_string = np.empty(shape=(function_enumeration_len,), dtype=object)
    blacklisted_terms_bool_arr_temp = blacklisted_terms_bool_arr.copy()
    # was uint32, but uint16 is sufficient for STRING v11, not using it for the foreground due to potential redundancy
    # or for "compare_samples" for the same reason --> keep the same
    funcEnum_count_background = np.zeros(shape=function_enumeration_len, dtype=np.dtype("uint32"))
    p_values = np.ones(shape=function_enumeration_len, dtype=np.dtype("float64"))
    p_values_corrected = np.ones(shape=function_enumeration_len, dtype=np.dtype("float64"))
    cond_multitest = np.zeros(function_enumeration_len, dtype=bool)
    cond_filter = np.ones(function_enumeration_len, dtype=bool)
    cond_PMIDs = np.zeros(function_enumeration_len, dtype=bool)
    cond_terms_reduced_with_ontology = np.zeros(function_enumeration_len, dtype=bool)
    background_ids_arr_of_string = np.empty(shape=(function_enumeration_len,), dtype=object)
    effectSizes = np.empty(function_enumeration_len, dtype=np.dtype("float64"))
    effectSizes.fill(np.nan)
    over_under_int_arr = np.zeros(function_enumeration_len, dtype=np.dtype("uint32")) # encoding of 1: "overrepresented", 2: "underrepresented", 0: "NaN"
    over_under_arr_of_string = np.empty(function_enumeration_len, np.dtype("U1"))
    return foreground_ids_arr_of_string, background_ids_arr_of_string, funcEnum_count_foreground, funcEnum_count_background, p_values, p_values_corrected, cond_multitest, blacklisted_terms_bool_arr_temp, cond_terms_reduced_with_ontology, cond_filter, cond_PMIDs, effectSizes, over_under_int_arr, over_under_arr_of_string

@boundscheck(False)
@wraparound(False)
cdef filter_parents_if_same_foreground(uint8[::1] blacklisted_terms_bool_arr_temp,
                                       cond_terms_reduced_with_ontology,
                                       dict lineage_dict_enum,
                                       df):
    """    
    potential speed up using C++ types for sets, BUT data is copied so profile 

    # distutils: language = c++    
    from libcpp.vector cimport vector
    from libcpp.set cimport set 
    """
    cdef:
        unsigned int term_enum, lineage_term
        # unsigned int lineage

    for group_terms in df.sort_values(["foreground_ids", "p_value", "hierarchical_level"], ascending=[True, True, False]).groupby("foreground_ids", sort=False).apply(lambda group: group["term_enum"].values):
        group_terms_set = set(group_terms)
        for term_enum in group_terms:
            if blacklisted_terms_bool_arr_temp[term_enum] == 0: # False
                cond_terms_reduced_with_ontology[term_enum] = True
                try:
                    lineage = lineage_dict_enum[term_enum] & group_terms_set # bitwise intersection
                except KeyError: # not in hierarchy (even though it should be, but some Reactome terms are inconsistent)
                    blacklisted_terms_bool_arr_temp[term_enum] = 1 # True
                    continue
                for lineage_term in lineage:
                    blacklisted_terms_bool_arr_temp[lineage_term] = 1 # True

def multiple_testing_per_entity_type(cond_etype, cond_multitest, p_values, p_values_corrected, indices_arr, num_total_tests):
    # select indices for given entity type and if multiple testing needs to be applied
    cond = cond_etype & cond_multitest
    # select p_values for BenjaminiHochberg
    p_values_2_BH = p_values[cond]
    # previously: num_total_tests = p_values_2_BH.shape[0]
    # select indices for BH
    indices_2_BH = indices_arr[cond]
    # sort p_values and remember indices sort order
    p_values_2_BH_sort_order = np.argsort(p_values_2_BH) # index positions of a reduced set
    indices_2_BH_of_superset = indices_2_BH[p_values_2_BH_sort_order]
    BenjaminiHochberg_cy(p_values, num_total_tests, p_values_corrected, indices_2_BH_of_superset)

def s_value(df, p_value_cutoff=0.05, KS_stat_cutoff=0.1, diff_proportions_cutoff=0.1):
    """
    calculate 's-value' type statistic in order to rank based on a combination of p-value and effect size
    for etypes -20, -25, and -26 (GOCC, BTO, and DOID) --> Common Language Effect Size
    for other etypes difference in ratios
    justification for cles_cutoff --> Kerby (https://doi.org/10.2466%2F11.IT.3.1) if the null is true the CLES is 50%
    justification for diff_proportions_cutoff --> unsure how to justify from lit. need be smaller than cles_cutoff
    --> changed from cles to KS_stat
    """
    min_pval = df["p_value"][df["p_value"] > 0].min()
    df["p_value_minlog"] = df["p_value"].apply(log_take_min_if_zero, args=(min_pval, ))
    df["s_value"] = 0.0
    cond_scores = df["etype"].isin([-20, -25, -26])
    p_value_cutoff = -1 * math.log10(p_value_cutoff) # test for values smaller than 0
    df["s_value"] = df["p_value_minlog"] * df["effectSize"]
    df = df.drop(columns=["p_value_minlog"])
    return df

def log_take_min_if_zero(val, min_pval):
    try:
        return -1*math.log10(val)
    except:
        return -1*math.log10(min_pval)

def limit_to_entity_types(limit_2_entity_type, function_enumeration_len, etype_cond_dict, funcEnum_count_foreground):
    if limit_2_entity_type is not None:
        cond_limit_2_entity_type = np.zeros(function_enumeration_len, dtype=bool)
        for cond_name in ["cond_" + etype[1:] for etype in limit_2_entity_type.split(";")]:
            try:
                cond_limit_2_entity_type |= etype_cond_dict[cond_name] # add other etypes
            except KeyError: # user provided etype can be mistyped of non-existent
                pass
        # set funcEnumAssociations to zero where cond_limit_2_entity_type is False
        funcEnum_count_foreground[~cond_limit_2_entity_type] = 0
        return cond_limit_2_entity_type # return bool arr of locations that should NOT be tested
    else:
        return np.ones(function_enumeration_len, dtype=bool)

def limit_to_go_subset(etype_cond_dict, go_slim_subset, goslimtype_2_cond_dict, funcEnum_count_foreground):
    if go_slim_subset is None:
        return funcEnum_count_foreground
    cond_GO_etypes = etype_cond_dict["cond_21"] | etype_cond_dict["cond_22"] | etype_cond_dict["cond_23"]
    cond = cond_GO_etypes != goslimtype_2_cond_dict[go_slim_subset] # select all GO terms that are not slim
    # set these to count 0
    funcEnum_count_foreground[cond] = 0
    return funcEnum_count_foreground

def add_funcEnums_2_dict(protein_ans_fg, ENSP_2_functionEnumArray_dict, ENSP_2_tuple_funcEnum_score_dict):
    ### add Protein 2 functionEnum info for JensenLabScore data to get foregroundIDs in DF
    for protein in protein_ans_fg:
        try: # sort is probably not necessary # potential speedup removing the sorting
            ENSP_2_functionEnumArray_dict[protein] = np.sort(np.concatenate((ENSP_2_tuple_funcEnum_score_dict[protein][0], ENSP_2_functionEnumArray_dict[protein])))
        except KeyError:
            pass # print("protein {} not in ENSP_2_tuple_funcEnum_score_dict".format(protein)) # --> simply not annotated with anything from textmining

def add_funcEnums_2_dict_CSC(protein_AN_set, ENSP_2_functionEnumArray_dict, ENSP_2_rowIndex_dict, CSR_ENSPencoding_2_FuncEnum):
    """
    rowIndex = ENSP_2_rowIndex_dict["128UP_DROME"]
    CSR_ENSPencoding_2_FuncEnum[rowIndex].indices # --> FunEnums_array == ENSP_2_tuple_funcEnum_score_dict["128UP_DROME"][0]
    CSR_ENSPencoding_2_FuncEnum[rowIndex].data # --> Scores_array == ENSP_2_tuple_funcEnum_score_dict[ensp][1]
    """
    for protein in protein_AN_set:
        try:
            rowIndex = ENSP_2_rowIndex_dict[protein]
        except KeyError:
            continue
        funcEnum_array = CSR_ENSPencoding_2_FuncEnum[rowIndex].indices
        ENSP_2_functionEnumArray_dict[protein] = np.sort(np.concatenate((funcEnum_array, ENSP_2_functionEnumArray_dict[protein])))

def replace_secondary_and_primary_IDs(ans_string, secondary_2_primary_dict, invert_dict=False):
    if invert_dict:
        dict_2_use = {v: k for k, v in secondary_2_primary_dict.items()}
    else:
        dict_2_use = secondary_2_primary_dict
    ids_2_return = []
    for id_ in ans_string.split(";"): # if proteinGroup
        if id_ in dict_2_use:
            ids_2_return.append(dict_2_use[id_])
        else:
            ids_2_return.append(id_)
    return ";".join(ids_2_return)

def add_protein_groups_to_ENSP_2_functionEnumArray_dict(ENSP_2_functionEnumArray_dict, all_unique_proteinGroups):
    """
    for all protein groups
    """
    for proteinGroup in all_unique_proteinGroups:
        if proteinGroup not in ENSP_2_functionEnumArray_dict:
            functionEnumArray_list = []
            for protein in proteinGroup.split(";"):
                try:
                    functionEnumArray_list.append(ENSP_2_functionEnumArray_dict[protein])
                except KeyError: # no functional annotation for given protein
                    pass
            try:
                ENSP_2_functionEnumArray_dict[proteinGroup] = reduce(np.union1d, functionEnumArray_list)
            except TypeError: # empty list
                #ENSP_2_functionEnumArray_dict[proteinGroup] = False #np.array(dtype=np.dtype("uint32"))
                pass
    return ENSP_2_functionEnumArray_dict

def slice_ScoresMatrix_for_given_ENSP(protein_AN_set, ENSP_2_rowIndex_dict, matrix):
    """
    produces 2D array
    number of rows corresponds to number of proteins if in ENSP_2_Score_dict
    number of columns corresponds to number of funcEnum of KS_etype; encoded as funcEnumIndex range(0, max(cond_KS_etypes)+1)
    """
    list_of_rowIndices = []
    for ENSP in protein_AN_set:
        try:
            rowIndex = ENSP_2_rowIndex_dict[ENSP]
        except KeyError:
            continue
        list_of_rowIndices.append(rowIndex)
    return matrix[list_of_rowIndices], list_of_rowIndices

def KolmogorovSmirnov_old_v2(foreground_n, background_n, funcEnum_2_scores_dict_fg, funcEnum_2_scores_dict_bg, p_values, cond_multitest, effectSizes, p_value_cutoff, funcEnum_count_foreground, funcEnum_count_background, over_under_int_arr, o_or_u_or_both_encoding, enrichment_method="genome"):
    for funcEnum, scores_fg in funcEnum_2_scores_dict_fg.items():
        scores_bg = funcEnum_2_scores_dict_bg[funcEnum]
        if o_or_u_or_both_encoding == 0:
            alternative = "two-sided"
        elif o_or_u_or_both_encoding == 1:
            alternative = "greater"
        elif o_or_u_or_both_encoding == 2:
            alternative = "less"
        statistic, pvalue, is_greater = KS_DBL(scores_fg, scores_bg, alternative=alternative) # statistic, pvalue = stats.ks_2samp(scores_fg, scores_bg)
        if pvalue <= p_value_cutoff:
            if o_or_u_or_both_encoding == 1 and is_greater: # overrepresented
                p_values[funcEnum] = pvalue
                effectSizes[funcEnum] = statistic
                over_under_int_arr[funcEnum] = 1
            elif o_or_u_or_both_encoding == 0: # both
                p_values[funcEnum] = pvalue
                effectSizes[funcEnum] = statistic
                if is_greater:
                    over_under_int_arr[funcEnum] = 1 # over
                else:
                    over_under_int_arr[funcEnum] = 2 # under
            elif o_or_u_or_both_encoding == 2 and not is_greater: # underrepresented
                p_values[funcEnum] = pvalue
                effectSizes[funcEnum] = statistic
                over_under_int_arr[funcEnum] = 2 # under
            else:
                pass
        cond_multitest[funcEnum] = True
        funcEnum_count_foreground[funcEnum] = len(scores_fg) # number of scores, important for BH (that this does not equal 0 or nan)
        funcEnum_count_background[funcEnum] = len(scores_bg)

def KS_DBL(data1, data2, alternative="two-sided", data2_sorted=False):
    data1 = np.sort(data1)
    if not data2_sorted: # genome comes presorted
        data2 = np.sort(data2)
    n1 = data1.shape[0]
    n2 = data2.shape[0]
    data_all = np.concatenate([data1, data2])
    # using searchsorted solves equal data problem
    cdf1 = np.searchsorted(data1, data_all, side='right') / n1
    cdf2 = np.searchsorted(data2, data_all, side='right') / n2
    cdf_diff = cdf1 - cdf2
    minS = -np.min(cdf_diff)
    maxS = np.max(cdf_diff)
    if alternative == "two-sided":
        D = max(minS, maxS)
    elif alternative == "greater":
        D = maxS
    elif alternative == "less":
        D = minS
    else:
        raise NotImplementedError
    p_value = math.exp(-2.0 * n1 * n2 * D * D / ( n1 + n2))
    if alternative != "two-sided":
        p_value /= 2
    if p_value > 1:
        p_value = 1
    if p_value < 0:
        p_value = 0
    if maxS > minS:
        is_greater = True
    else:
        is_greater = False
    return D, p_value, is_greater

def KS_DBL_adapted_from_Christian(fg_values, bg_values):
    fg_values = sorted(fg_values)
    bg_values = sorted(bg_values)
    fg_size = len(fg_values)
    bg_size = len(bg_values)
    n1 = fg_size
    n2 = bg_size
    n1_plus_n2 = n1 + n2
    n1_times_n2_times_mintwo = -2.0 * n1 * n2
    D_max = 0.0
    fg_rank, bg_rank = 0, 0
    while fg_rank < fg_size:
        fg_cumulative = fg_rank / fg_size
        fg_val = fg_values[fg_rank]
        for bg_rank_temp, bg_val in enumerate(bg_values[bg_rank:], 0):
            if fg_val <= bg_val:
                bg_rank = bg_rank_temp + bg_rank
                break
        bg_cumulative = (bg_rank + 1) / bg_size
        D_current = abs(fg_cumulative - bg_cumulative)
        if D_current > D_max:
            D_max = D_current
        fg_rank += 1
        fg_cumulative = fg_rank / fg_size
        D_current = abs(fg_cumulative - bg_cumulative)
        if D_current > D_max:
            D_max = D_current
    pvalue = math.exp(n1_times_n2_times_mintwo * D_max * D_max / n1_plus_n2)
    return D_max, pvalue

def filter_stuff(args_dict, protein_ans_fg, p_values_corrected, foreground_ids_arr_of_string, funcEnum_count_foreground, year_arr, p_values, indices_arr, ENSP_2_functionEnumArray_dict, cond_filter, etype_cond_dict, cond_PMIDs, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, over_under_int_arr):
    FDR_cutoff, p_value_cutoff = args_dict["FDR_cutoff"], args_dict["p_value_cutoff"]
    cond_filter = (p_values_corrected <= FDR_cutoff) & (p_values <= p_value_cutoff)
    ### remove terms with only a single annotation
    if args_dict["filter_foreground_count_one"] is True:
        cond_filter &= funcEnum_count_foreground > 1
    else:  # remove terms without any annotation
        cond_filter &= funcEnum_count_foreground > 0

    ### overrepresented/underrepresented/both
    o_or_u_or_both_encoding = args_dict["o_or_u_or_both_encoding"]
    if o_or_u_or_both_encoding == 1: # overrepresented
        cond_o_or_u_or_both = over_under_int_arr == 1
    elif o_or_u_or_both_encoding == 2: # underrepresented
        cond_o_or_u_or_both = over_under_int_arr == 2
    elif o_or_u_or_both_encoding == 0: # both
        cond_o_or_u_or_both = over_under_int_arr > 0
    else:
        pass # should not happen
    cond_filter &= cond_o_or_u_or_both
    filter_PMID_top_n = args_dict["filter_PMID_top_n"]
    if filter_PMID_top_n is not None:
        cond_PMID_2_filter = cond_filter & etype_cond_dict["cond_56"]  # -56
        df_PMID = pd.DataFrame({"foreground_count": funcEnum_count_foreground[cond_PMID_2_filter].view(), "year": year_arr[cond_PMID_2_filter].view(), "p_value": p_values[cond_PMID_2_filter].view(), "FDR": p_values_corrected[cond_PMID_2_filter].view(), "indices_arr": indices_arr[cond_PMID_2_filter].view()})
        indices_PMID = df_PMID.sort_values(["FDR", "p_value", "year", "foreground_count"], ascending=[True, True, False, False])["indices_arr"].values[:filter_PMID_top_n]
        for index_ in indices_PMID:
            cond_PMIDs[index_] = True
    else:  # since no filtering use all PMIDs
        cond_PMIDs = cond_filter & etype_cond_dict["cond_56"]
    cond_etypes_with_ontology_filtered = cond_etypes_with_ontology & cond_filter  # {-21, -22, -23, -51, -57}
    # entity_types_with_ontology = {-20, -21, -22, -23, -25, -26, -51, -57} # Interpro has ontology, but omitted here to turn off filter_parents functionality
    cond_etypes_rem_foreground_ids_filtered = cond_etypes_rem_foreground_ids & cond_filter  # remaining etypes -52, -53, -54, -55
    cond_IDs_2_query = (cond_PMIDs | cond_etypes_with_ontology_filtered | cond_etypes_rem_foreground_ids_filtered)
    ### get foreground IDs of relevant subset --> array for entire data set
    ## exclude TextMining KS functionEnumerations since these are probably not very informative and we need performance --> don't exclude
#     if not KS_etypes_FG_IDs:
#         cond_IDs_2_query = cond_IDs_2_query & ~cond_KS_etypes # commented on purpose since STRING needs these
    funcEnum_indices_for_IDs = indices_arr[cond_IDs_2_query]
    foreground_ids_arr_of_string = map_funcEnum_2_ENSPs(protein_ans_fg, ENSP_2_functionEnumArray_dict, funcEnum_indices_for_IDs, foreground_ids_arr_of_string)
#     if not KS_etypes_FG_IDs:
#         foreground_ids_arr_of_string[cond_KS_etypes] = "" # commented out for STRING
    return foreground_ids_arr_of_string, funcEnum_indices_for_IDs, cond_etypes_with_ontology_filtered, cond_etypes_rem_foreground_ids_filtered, cond_filter


@boundscheck(False)
@wraparound(False)
cdef int KolmogorovSmirnov_sparse_cy(FunctionEnum_2_Scores_dict, unsigned int foreground_n, unsigned int background_n, unsigned int [::1] fg_scores_matrix_data, int [::1] fg_scores_matrix_indptr, unsigned int [::1] bg_scores_matrix_data, int [::1] bg_scores_matrix_indptr, double[::1] p_values, cond_multitest, double[::1] effectSizes, double p_value_cutoff, unsigned int[::1] funcEnum_count_foreground, unsigned int[::1] funcEnum_count_background, unsigned int[::1] over_under_int_arr, unsigned int o_or_u_or_both_encoding, enrichment_method, filter_foreground_count_one, debug=False):
    cdef:
        int bg_rank_temp
        unsigned int median_index, num_half_bg, num_half_fg, num_zeros_2_fill_bg, num_zeros_2_fill_fg, fg_size_plus_bg_size, funcEnum, bg_index, len_fg_scores_matrix_indptr, index_col_start_fg, index_col_stop_fg, index_col_start_bg, index_col_stop_bg, num_fg_vals, num_bg_vals, fg_val, bg_val, fg_rank, bg_rank
        double fg_size_times_bg_size_times_mintwo, pvalue, D_max_abs, D_current_abs, D_current_absfg_cumulative, bg_cumulative, median_fg, median_bg
        unsigned int[::1] fg_values, bg_values
    fg_size_plus_bg_size = foreground_n + background_n
    fg_size_times_bg_size_times_mintwo = -2.0 * foreground_n * background_n
    len_fg_scores_matrix_indptr = fg_scores_matrix_indptr.shape[0]
    for funcEnum in range(len_fg_scores_matrix_indptr - 1):
        index_col_start_fg = fg_scores_matrix_indptr[funcEnum]
        index_col_stop_fg = fg_scores_matrix_indptr[funcEnum + 1]
        if index_col_start_fg == index_col_stop_fg:
            continue # column is empty
        elif filter_foreground_count_one and (index_col_stop_fg - index_col_start_fg) == 1:
            continue
        else:
            fg_values = fg_scores_matrix_data[index_col_start_fg:index_col_stop_fg]

        if enrichment_method == "genome": # is pre-sorted
            try:
                bg_values = FunctionEnum_2_Scores_dict[funcEnum]
            except KeyError:
                continue
            num_bg_vals = bg_values.shape[0]
            if num_bg_vals == 0:
                continue
        else:
            index_col_start_bg = bg_scores_matrix_indptr[funcEnum]
            index_col_stop_bg = bg_scores_matrix_indptr[funcEnum + 1]
            if index_col_start_bg == index_col_stop_bg:
                continue # column is empty
            else:
                bg_values = bg_scores_matrix_data[index_col_start_bg:index_col_stop_bg]
            bg_values = np.sort(bg_values)
            num_bg_vals = bg_values.shape[0]
        fg_values = np.sort(fg_values)
        num_fg_vals = fg_values.shape[0]
        num_zeros_2_fill_fg = foreground_n - num_fg_vals
        num_zeros_2_fill_bg = background_n - num_bg_vals
        fg_rank, bg_rank, D_max_abs = 0, 0, 0
        while fg_rank < num_fg_vals:
            fg_val = fg_values[fg_rank]
            fg_cumulative = (fg_rank + num_zeros_2_fill_fg) / foreground_n
            bg_rank_temp = 0
            for bg_index in range(bg_rank, num_bg_vals):
                bg_val = bg_values[bg_index]
                if fg_val <= bg_val:
                    bg_rank = bg_rank_temp + bg_rank
                    break
                bg_rank_temp += 1

            bg_cumulative = (bg_rank + num_zeros_2_fill_bg + 1) / background_n
            D_current_abs = abs(fg_cumulative - bg_cumulative)
            if D_current_abs > D_max_abs:
                D_max_abs = D_current_abs

            fg_rank += 1
            fg_cumulative = (fg_rank + num_zeros_2_fill_fg) / foreground_n
            D_current_abs = abs(fg_cumulative - bg_cumulative)
            if D_current_abs > D_max_abs:
                D_max_abs = D_current_abs
        pvalue = math.exp(fg_size_times_bg_size_times_mintwo * D_max_abs * D_max_abs / fg_size_plus_bg_size)
        if o_or_u_or_both_encoding != 0:
            pvalue /= 2
        num_half_fg = int(round((num_fg_vals + num_zeros_2_fill_fg)/2)) # index at half of fg
        if num_half_fg > num_zeros_2_fill_fg:
            median_index = int(num_half_fg - num_zeros_2_fill_fg)
            median_fg = fg_values[median_index]
        else:
            median_fg = 0
        num_half_bg = int(round((num_bg_vals + num_zeros_2_fill_bg)/2))
        if num_half_bg > num_zeros_2_fill_bg:
            median_index = int(num_half_bg - num_zeros_2_fill_bg)
            median_bg = bg_values[median_index]
        else:
            median_bg = 0
        is_greater = median_fg > median_bg # since rank based this is inverted

        if pvalue <= p_value_cutoff:
            p_values[funcEnum] = pvalue
            effectSizes[funcEnum] = D_max_abs
            if is_greater: # overrepresented
                over_under_int_arr[funcEnum] = 1
            else: # underrepresented
                over_under_int_arr[funcEnum] = 2
        cond_multitest[funcEnum] = True
        funcEnum_count_foreground[funcEnum] = num_fg_vals
        funcEnum_count_background[funcEnum] = num_bg_vals
    return 0

def KolmogorovSmirnov_scipy(foreground_n, background_n, funcEnum_2_scores_dict_fg, funcEnum_2_scores_dict_bg, p_values, cond_multitest, effectSizes, p_value_cutoff, funcEnum_count_foreground, funcEnum_count_background, over_under_int_arr, o_or_u_or_both_encoding, fill_zeros=True):
    for funcEnum, scores_fg in funcEnum_2_scores_dict_fg.items():
        funcEnum_count_foreground[funcEnum] = sum(scores_fg > 0) # len(scores_fg) # don't count Zeros
        # number of scores, important for BH (that this does not equal 0 or nan)
        scores_fg = list(scores_fg) # already filled with 0
        try:
            scores_bg = funcEnum_2_scores_dict_bg[funcEnum]
            funcEnum_count_background[funcEnum] = sum(scores_bg > 0)
            scores_bg = list(scores_bg)
        except KeyError: # funcEnum not in background
            continue
        len_scores_fg = len(scores_fg)
        if fill_zeros:
            number_of_zeros_2_fill = foreground_n - len_scores_fg
            if number_of_zeros_2_fill > 0:
                scores_fg = [0]*number_of_zeros_2_fill + scores_fg
        len_scores_bg = len(scores_bg)
        if fill_zeros:
            number_of_zeros_2_fill = background_n - len_scores_bg
            if number_of_zeros_2_fill > 0:
                scores_bg = [0]*number_of_zeros_2_fill + scores_bg
        statistic, pvalue = stats.ks_2samp(scores_fg, scores_bg, alternative="two-sided", mode="asymp")
        if pvalue <= p_value_cutoff:
            p_values[funcEnum] = pvalue
            effectSizes[funcEnum] = statistic
            is_greater = np.median(scores_fg) > np.median(scores_bg)
            ### use all values since test is two-tailed (and multiple testing had to be done)
            # filter for overrepresented/underrepresented terms
            if is_greater: # overrepresented
                over_under_int_arr[funcEnum] = 1
            else:
                over_under_int_arr[funcEnum] = 2 # under
        cond_multitest[funcEnum] = True

@boundscheck(False)
@wraparound(False)
cdef int KolmogorovSmirnov_sparse_scipy(FunctionEnum_2_Scores_dict, unsigned int foreground_n, unsigned int background_n, unsigned int [::1] fg_scores_matrix_data, int [::1] fg_scores_matrix_indptr, unsigned int [::1] bg_scores_matrix_data, int [::1] bg_scores_matrix_indptr, double[::1] p_values, cond_multitest, double[::1] effectSizes, double p_value_cutoff, unsigned int[::1] funcEnum_count_foreground, unsigned int[::1] funcEnum_count_background, unsigned int[::1] over_under_int_arr, unsigned int o_or_u_or_both_encoding, enrichment_method, filter_foreground_count_one, debug=False):
    cdef:
        int bg_rank_temp, n1, n2
        unsigned int median_index, num_half_bg, num_half_fg, num_zeros_2_fill_bg, num_zeros_2_fill_fg, fg_size_plus_bg_size, funcEnum, bg_index, len_fg_scores_matrix_indptr, index_col_start_fg, index_col_stop_fg, index_col_start_bg, index_col_stop_bg, num_fg_vals, num_bg_vals, fg_val, bg_val, fg_rank, bg_rank
        double fg_size_times_bg_size_times_mintwo, p_value, D, bg_cumulative, median_fg, median_bg
        unsigned int[::1] fg_values, bg_values
    fg_size_plus_bg_size = foreground_n + background_n
    fg_size_times_bg_size_times_mintwo = -2.0 * foreground_n * background_n
    len_fg_scores_matrix_indptr = fg_scores_matrix_indptr.shape[0]
    for funcEnum in range(len_fg_scores_matrix_indptr - 1):
        index_col_start_fg = fg_scores_matrix_indptr[funcEnum]
        index_col_stop_fg = fg_scores_matrix_indptr[funcEnum + 1]
        if index_col_start_fg == index_col_stop_fg:
            continue # column is empty
        elif filter_foreground_count_one and (index_col_stop_fg - index_col_start_fg) == 1:
            continue
        else:
            fg_values = fg_scores_matrix_data[index_col_start_fg:index_col_stop_fg]

        if enrichment_method == "genome": # is pre-sorted
            try:
                bg_values = FunctionEnum_2_Scores_dict[funcEnum]
            except KeyError:
                continue
            num_bg_vals = bg_values.shape[0]
            if num_bg_vals == 0:
                continue
        else:
            index_col_start_bg = bg_scores_matrix_indptr[funcEnum]
            index_col_stop_bg = bg_scores_matrix_indptr[funcEnum + 1]
            if index_col_start_bg == index_col_stop_bg:
                continue # column is empty
            else:
                bg_values = bg_scores_matrix_data[index_col_start_bg:index_col_stop_bg]
            bg_values = np.sort(bg_values)
            num_bg_vals = bg_values.shape[0]
        fg_values = np.sort(fg_values)
        num_fg_vals = fg_values.shape[0]
        num_zeros_2_fill_fg = foreground_n - num_fg_vals
        num_zeros_2_fill_bg = background_n - num_bg_vals
        fg_values_with_zeros = np.concatenate((np.zeros((num_zeros_2_fill_fg,), dtype=variables.dtype_TM_score), fg_values))
        bg_values_with_zeros = np.concatenate((np.zeros((num_zeros_2_fill_bg,), dtype=variables.dtype_TM_score), bg_values))
        n1 = fg_values_with_zeros.shape[0]
        n2 = bg_values_with_zeros.shape[0]
        data_all = np.concatenate([fg_values_with_zeros, bg_values_with_zeros])
        # using searchsorted solves equal data problem
        cdf1 = np.searchsorted(fg_values_with_zeros, data_all, side='right') / n1
        cdf2 = np.searchsorted(bg_values_with_zeros, data_all, side='right') / n2
        cdf_diff = cdf1 - cdf2
        minS = -np.min(cdf_diff)
        maxS = np.max(cdf_diff)
        D = max(minS, maxS)
        p_value = math.exp(-2.0 * n1 * n2 * D * D / ( n1 + n2)) * 2
        if p_value > 1:
            p_value = 1
        if p_value < 0:
            p_value = 0
        if maxS < minS: # inverted since scores not ranks
            is_greater = True
        else:
            is_greater = False
        if p_value <= p_value_cutoff:
            p_values[funcEnum] = p_value
            effectSizes[funcEnum] = D
            if is_greater: # overrepresented
                over_under_int_arr[funcEnum] = 1
            else: # underrepresented
                over_under_int_arr[funcEnum] = 2
        cond_multitest[funcEnum] = True
        funcEnum_count_foreground[funcEnum] = num_fg_vals
        funcEnum_count_background[funcEnum] = num_bg_vals
    return 0

def run_characterize_foreground_cy(ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=False):
    if not low_memory:
        # ENSP_2_functionEnumArray_dict, year_arr, hierlevel_arr, entitytype_arr, functionalterm_arr, indices_arr, description_arr, category_arr, etype_2_minmax_funcEnum, function_enumeration_len, etype_cond_dict, etype_2_num_functions_dict, taxid_2_proteome_count, taxid_2_tuple_funcEnum_index_2_associations_counts, lineage_dict_enum, blacklisted_terms_bool_arr, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, kegg_taxid_2_acronym_dict, goslimtype_2_cond_dict, ENSP_2_rowIndex_dict, rowIndex_2_ENSP_dict, CSC_ENSPencoding_2_FuncEnum, CSR_ENSPencoding_2_FuncEnum, Taxid_2_FunctionEnum_2_Scores_dict = static_preloaded_objects
        ENSP_2_functionEnumArray_dict, year_arr, hierlevel_arr, entitytype_arr, functionalterm_arr, indices_arr, description_arr, category_arr, etype_2_minmax_funcEnum, function_enumeration_len, etype_cond_dict, etype_2_num_functions_dict, taxid_2_proteome_count, taxid_2_tuple_funcEnum_index_2_associations_counts, lineage_dict_enum, blacklisted_terms_bool_arr, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, kegg_taxid_2_acronym_dict, goslimtype_2_cond_dict = static_preloaded_objects
    else:  # missing: ENSP_2_functionEnumArray_dict
        # year_arr, hierlevel_arr, entitytype_arr, functionalterm_arr, indices_arr, description_arr, category_arr, etype_2_minmax_funcEnum, function_enumeration_len, etype_cond_dict, etype_2_num_functions_dict, taxid_2_proteome_count, taxid_2_tuple_funcEnum_index_2_associations_counts, lineage_dict_enum, blacklisted_terms_bool_arr, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, kegg_taxid_2_acronym_dict, goslimtype_2_cond_dict, ENSP_2_rowIndex_dict, rowIndex_2_ENSP_dict, CSC_ENSPencoding_2_FuncEnum, CSR_ENSPencoding_2_FuncEnum, Taxid_2_FunctionEnum_2_Scores_dict = static_preloaded_objects
        year_arr, hierlevel_arr, entitytype_arr, functionalterm_arr, indices_arr, description_arr, category_arr, etype_2_minmax_funcEnum, function_enumeration_len, etype_cond_dict, etype_2_num_functions_dict, taxid_2_proteome_count, taxid_2_tuple_funcEnum_index_2_associations_counts, lineage_dict_enum, blacklisted_terms_bool_arr, cond_etypes_with_ontology, cond_etypes_rem_foreground_ids, kegg_taxid_2_acronym_dict, goslimtype_2_cond_dict = static_preloaded_objects
    foreground_ids_arr_of_string, background_ids_arr_of_string, funcEnum_count_foreground, funcEnum_count_background, p_values, p_values_corrected, cond_multitest, blacklisted_terms_bool_arr_temp, cond_terms_reduced_with_ontology, cond_filter, cond_PMIDs, effectSizes, over_under_int_arr, over_under_arr_of_string = preloaded_objects_per_analysis
    em = ui.enrichment_method
    foreground_n = ui.get_foreground_n()
    args_dict = ui.args_dict
    filter_foreground_count_one = args_dict["filter_foreground_count_one"]
    cols_2_return_sort_order = variables.cols_sort_order_characterize_foreground[:]

    protein_ans_fg = ui.get_foreground_an_set()
    if low_memory:
        ENSP_2_functionEnumArray_dict = query.get_functionEnumArray_from_proteins(ui.get_all_individual_AN(), dict_2_array=True)
    ### add protein groups to ENSP_2_functionEnumArray_dict
    ENSP_2_functionEnumArray_dict = add_protein_groups_to_ENSP_2_functionEnumArray_dict(ENSP_2_functionEnumArray_dict, ui.get_all_unique_proteinGroups())

    ## count foreground
    count_all_terms(ENSP_2_functionEnumArray_dict, protein_ans_fg, funcEnum_count_foreground)

    ## limit to given entity types
    cond_limit_2_entity_type = limit_to_entity_types(args_dict["limit_2_entity_type"], function_enumeration_len, etype_cond_dict, funcEnum_count_foreground)
    limit_to_go_subset(etype_cond_dict, args_dict["go_slim_subset"], goslimtype_2_cond_dict, funcEnum_count_foreground)

#     ### Jensenlab Scores KS test
#     cond_KS_etypes = etype_cond_dict["cond_25"] | etype_cond_dict["cond_26"] | etype_cond_dict["cond_20"]
#     funcEnums_2_include_set = set(indices_arr[cond_KS_etypes & cond_limit_2_entity_type])

    # orig
#     if orig:
#         funcEnum_2_scores_dict_fg = collect_scores_per_term_characterize_foreground(protein_ans_fg, ENSP_2_tuple_funcEnum_score_dict, funcEnums_2_include_set, score_cutoff=args_dict["score_cutoff"])
#         for funcEnum, scores_fg in funcEnum_2_scores_dict_fg.items():
#             funcEnum_count_foreground[funcEnum] = len(scores_fg)
    # new CSC version
#     else:

#     fg_scores_matrix, list_of_rowIndices_fg = slice_ScoresMatrix_for_given_ENSP(protein_ans_fg, ENSP_2_rowIndex_dict, CSC_ENSPencoding_2_FuncEnum)
#     fg_scores_matrix_data = fg_scores_matrix.data
#     fg_scores_matrix_indptr = fg_scores_matrix.indptr
#     if fg_scores_matrix_data.size > 0:
#         set_fg_counts(fg_scores_matrix_data, fg_scores_matrix_indptr, funcEnum_count_foreground, filter_foreground_count_one)
#         # add_funcEnums_2_dict(protein_ans_fg, ENSP_2_functionEnumArray_dict, ENSP_2_tuple_funcEnum_score_dict)
#         add_funcEnums_2_dict_CSC(protein_ans_fg, ENSP_2_functionEnumArray_dict, ENSP_2_rowIndex_dict, CSR_ENSPencoding_2_FuncEnum)

    ### calc ratio in foreground, count foreground / len(protein_ans)
    ratio_in_foreground = funcEnum_count_foreground / foreground_n

    ### concatenate filtered results
    if filter_foreground_count_one:
        cond_2_return = funcEnum_count_foreground > 1
    else:
        cond_2_return = funcEnum_count_foreground >= 1

    ### limit PMID results
    filter_PMID_top_n = args_dict["filter_PMID_top_n"]
    if filter_PMID_top_n is not None:
        cond_PMID_2_filter = cond_2_return & etype_cond_dict["cond_56"]
        df_PMID = pd.DataFrame({"foreground_count": funcEnum_count_foreground[cond_PMID_2_filter].view(), "year": year_arr[cond_PMID_2_filter].view(), "indices_arr": indices_arr[cond_PMID_2_filter].view()})
        indices_PMID = df_PMID.sort_values(["foreground_count", "year"], ascending=[False, False])["indices_arr"].values[:filter_PMID_top_n]
        # set all PMIDs to False and then include only those that were selected
        cond_2_return[etype_cond_dict["cond_56"]] = False
        for index_ in indices_PMID:
            cond_2_return[index_] = True
    ### exclude blacklisted terms
#     print(len(cond_2_return), sum(cond_2_return))    
    cond_2_return[blacklisted_terms_bool_arr > 0] = False
#     print(blacklisted_terms_bool_arr)
#     print(len(cond_2_return), sum(cond_2_return))

    try:
        privileged = args_dict["privileged"]
    except KeyError:
        privileged = False
    if not privileged:
        # remove KEGG unless privileged
        cond_kegg = etype_cond_dict["cond_52"]
        cond_2_return = cond_2_return & ~cond_kegg

    funcEnum_indices_for_IDs = indices_arr[cond_2_return]
    foreground_ids_arr_of_string = map_funcEnum_2_ENSPs(protein_ans_fg, ENSP_2_functionEnumArray_dict, funcEnum_indices_for_IDs, foreground_ids_arr_of_string)
    if not low_memory:
        df_2_return = pd.DataFrame({"term": functionalterm_arr[cond_2_return].view(),
                                    "hierarchical_level": hierlevel_arr[cond_2_return].view(),
                                    "category": category_arr[cond_2_return].view(),
                                    "etype": entitytype_arr[cond_2_return].view(),
                                    "description": description_arr[cond_2_return].view(),
                                    "year": year_arr[cond_2_return].view(),
                                    "ratio_in_FG": ratio_in_foreground[cond_2_return].view(),
                                    "FG_ids": foreground_ids_arr_of_string[cond_2_return].view(),
                                    "FG_count": funcEnum_count_foreground[cond_2_return].view()})
    else:
        df_2_return = pd.DataFrame({"term": functionalterm_arr[cond_2_return].view(),
                                    "hierarchical_level": hierlevel_arr[cond_2_return].view(),
                                    "etype": entitytype_arr[cond_2_return].view(),
                                    "year": year_arr[cond_2_return].view(),
                                    "ratio_in_FG": ratio_in_foreground[cond_2_return].view(),
                                    "FG_IDs": foreground_ids_arr_of_string[cond_2_return].view(),
                                    "FG_count": funcEnum_count_foreground[cond_2_return].view(),
                                    "funcEnum": indices_arr[cond_2_return].view()})
        df_2_return["category"] = df_2_return["etype"].apply(lambda etype: variables.entityType_2_functionType_dict[etype])
        funcEnum_2_description_dict = query.get_function_description_from_funcEnum(indices_arr[cond_2_return].tolist())
        df_2_return["description"] = df_2_return["funcEnum"].apply(lambda funcEnum: funcEnum_2_description_dict[funcEnum])
    #cols_2_return_sort_order = ['etype', 'term', 'hierarchical_level', 'description', 'year','ratio_in_FG', 'FG_count', 'FG_n', 'FG_IDs', 'funcEnum', 'category']
    df_2_return = ui.translate_primary_back_to_secondary(df_2_return)
    df_2_return["FG_n"] = foreground_n
    # rank everything correctly except PMIDs, "year"-column will only affect PMIDs
    df_2_return = df_2_return.sort_values(["etype", "year", "FG_count"], ascending=[True, False, False]).reset_index(drop=True)
    cond_PMIDs = df_2_return["etype"] == -56
    df_2_return.loc[~cond_PMIDs, "rank"] = df_2_return[~cond_PMIDs].groupby("etype")["FG_count"].rank(ascending=False, method="first").fillna(value=df_2_return.shape[0])
    df_2_return.loc[cond_PMIDs, "rank"] = df_2_return[cond_PMIDs].groupby("etype")["year"].rank(ascending=False, method="first").fillna(value=df_2_return.shape[0])
    df_2_return["rank"] = df_2_return["rank"].astype(int)
    return df_2_return[cols_2_return_sort_order]

## run ze function!

In [13]:
# fn_userinput = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/Example_Yeast_acetylation_abundance_correction.txt"
# os.path.basename(fn_userinput)

In [12]:
from importlib import reload
reload(userinput)
ENSP_2_tuple_funcEnum_score_dict = None
#######################
### examples
# foreground_input = ["HBA_HUMAN", "HBB_HUMAN", "HBD_HUMAN", "HBE_HUMAN"] # example 1: Human hemoglobin, genome
# example 2: Yeast acetylation, abundance_correction. Example_1_Yeast_acetylation_abundance_correction.txt
# example 3: compare_samples
# human plasma liver cirrhosis Lili Niu
# foreground_input = ['P05062;A0A024R145;A8K430;A0A087WXX2;Q8NHT3', 'Q08380;B4DVE1;A0A0S2Z3Y1;B3KP88;B4DDG4;B4DWA8;B4DI70', 'P04004;D9ZGG2;B7Z553', 'O95445', 'P01833', 'P43652', 'Q5SRP5']
# mouse Insulin (STRING network )
# foreground_input = ['10090.ENSMUSP00000005671', '10090.ENSMUSP00000020846', '10090.ENSMUSP00000022921', '10090.ENSMUSP00000028252', '10090.ENSMUSP00000056668', '10090.ENSMUSP00000061877', '10090.ENSMUSP00000084464', '10090.ENSMUSP00000088837', '10090.ENSMUSP00000099787', '10090.ENSMUSP00000099862', '10090.ENSMUSP00000104298']
# mouse interferon (STRING network)
# foreground_input = ['10090.ENSMUSP00000001036', '10090.ENSMUSP00000023689', '10090.ENSMUSP00000023693', '10090.ENSMUSP00000038121', '10090.ENSMUSP00000056720', '10090.ENSMUSP00000066743', '10090.ENSMUSP00000092581', '10090.ENSMUSP00000099842', '10090.ENSMUSP00000100872', '10090.ENSMUSP00000120525', '10090.ENSMUSP00000127921']
#Q9R117\nP33896\nO35664\nO35716\nP01575\nP42225\nP07351\nP52332\nQ9WVL2\nQ61179\nQ61716
# foreground_input = ["ADH1_YEAST", "PDC1_YEAST", "PFKA1_YEAST"]
# foreground_input = ["PGM1_YEAST", "G6PI_YEAST", "PMG2_YEAST", "CISY2_YEAST"]
############
contiguous = True
foreground_n = 300
# foreground_input = sorted(get_random_human_ENSP(foreground_n, joined_for_web=False, contiguous=contiguous, UniProt_ID=True))
# foreground_input = ['MEF2A_HUMAN', 'MEF2B_HUMAN', 'MEF2C_HUMAN', 'MEF2D_HUMAN', 'MEFV_HUMAN', 'MEG10_HUMAN', 'MEG11_HUMAN', 'MEGF6_HUMAN', 'MEGF8_HUMAN', 'MEGF9_HUMAN', 'MEI1_HUMAN', 'MEI4_HUMAN', 'MEIG1_HUMAN', 'MEIKN_HUMAN', 'MEIOB_HUMAN', 'MEIOC_HUMAN', 'MEIS1_HUMAN', 'MEIS2_HUMAN', 'MEIS3_HUMAN', 'MELK_HUMAN']
from_file = True # read user input from file
fn_userinput = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/Example_Yeast_acetylation_abundance_correction.txt"
fn_userinput = r"/Users/dblyon/Downloads/agotoolquestions/ClpP2up_KEimputed_aGOtool.txt"
# fn_userinput = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/Example_1_Yeast_acetylation_foreground_only.txt"
# fn_userinput = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/Example_1.1_Yeast_acetylation_without_abundance.txt"
# foreground_input = ["P69905"]
# foreground_input = corona

enrichment_method = "abundance_correction" # "" "abundance_correction" "compare_samples" "genome" "compare_groups"
args_dict = {}
args_dict["enrichment_method"] = enrichment_method
args_dict["taxid"] = 9606 # 9606 # 559292 Yeast
args_dict["FDR_cutoff"] = 0.05
args_dict["p_value_cutoff"] = 0.01
args_dict["limit_2_entity_type"] = None # "-20;-25;-26" #"-21;-22;-23;-51;-52;-53;-54;-55;-56-57;-58"
args_dict["filter_PMID_top_n"] = 100
args_dict["filter_foreground_count_one"] = True
args_dict["filter_parents"] = True
args_dict["go_slim_subset"] = None # "generic"
args_dict["o_or_u_or_both"] = "both" # "both" "underrepresented" "overrepresented"
args_dict["multiple_testing_per_etype"] = True
args_dict["privileged"] = True
args_dict["score_cutoff"] = 0
# args_dict["foreground_replicates"] = 10
# args_dict["background_replicates"] = 10
taxid = args_dict["taxid"]
debug = False
profile = False
simplified_output = False
args_dict["simplified_output"] = simplified_output
args_dict["STRING_beta"] = False
# KS_method = "sparse_scipy" # {"scipy", "cy", "sparse_scipy"}
background_n = 300
contiguous = True
# background_input = sorted(get_random_human_ENSP(background_n, joined_for_web=False, contiguous=contiguous)) # background_input = ENSPs_homo
# background_input = query.get_proteins_of_taxid(taxid, read_from_flat_files=True)
# background_input = None
if from_file:
    ui = userinput.Userinput(pqo, fn_userinput, args_dict=args_dict)
else:
    ui = userinput.Userinput(pqo, fn=None, foreground_string=userinput.stringify_for_Userinput(foreground_input), background_string=userinput.stringify_for_Userinput(background_input), args_dict=args_dict)
ncbi = pqo.ncbi
preloaded_objects_per_analysis = pqo.get_preloaded_objects_per_analysis()

if profile:
    profile = line_profiler.LineProfiler(calc_pvalues_v2) # run_enrichment_cy, KolmogorovSmirnov_sparse_cy, KolmogorovSmirnov_sparse_cy_genome
    profile.runcall(run_enrichment_cy, ENSP_2_tuple_funcEnum_score_dict, ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug, KS_method=KS_method)
    profile.print_stats()
else:
    if enrichment_method == "characterize_foreground":
        df = run_characterize_foreground_cy(ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True)
        df = df.sort_values(["etype", "FG_count"], ascending=[False, False])
    else:
        if debug:
            df = run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug)
        else:
            df = run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug)
            df = df.sort_values(["etype", "rank"], ascending=[False, True])
            print(df.shape)
            print(df.groupby("category")["term"].count())

userinput 107 something wrong


AttributeError: 'Userinput' object has no attribute 'foreground'

In [19]:
reload(userinput)

<module 'userinput' from '/Users/dblyon/modules/cpr/agotool/app/python/userinput.py'>

In [35]:
import random
taxid = random.choice(query.get_taxids())
background = query.get_proteins_of_taxid(taxid)
foreground = random.sample(background, 200)
intensity = [str(ele) for ele in np.random.normal(size=len(background))]

In [57]:
enrichment_method = "abundance_correction"
args_dict = {}
args_dict["enrichment_method"] = enrichment_method
args_dict["taxid"] = taxid
args_dict["FDR_cutoff"] = 1
args_dict["p_value_cutoff"] = 1
args_dict["limit_2_entity_type"] = None 
args_dict["filter_PMID_top_n"] = 100
args_dict["filter_foreground_count_one"] = True
args_dict["filter_parents"] = True
args_dict["go_slim_subset"] = None
args_dict["o_or_u_or_both_encoding"] = 0
args_dict["multiple_testing_per_etype"] = True
args_dict["privileged"] = True
args_dict["score_cutoff"] = 0
args_dict["num_bins"] = 100
args_dict["foreground_n"] = None
args_dict["background_n"] = None
args_dict["simplified_output"] = False
args_dict["foreground"] = "%0d".join(foreground)
args_dict["background"] = "%0d".join(background)
args_dict["background_intensity"] = "%0d".join(intensity)
preloaded_objects_per_analysis = pqo.get_preloaded_objects_per_analysis()
ui = userinput.REST_API_input(pqo, args_dict=args_dict)
df = run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug)
assert (df["FG_n"] == df["BG_n"]).all()
assert (df["FG_count"] <= df["FG_n"]).all()
assert (df["BG_count"] <= df["BG_n"]).all()
assert (df["FG_count"] <= df["BG_count"]).all()

A0A2L0PPD5_VITS1 0.15 [ 5378  5401  5880  5881  6497  6498  7135  8224  8348  9324  9325 12485
 13034 13347 13709 15735 24006 24207 24453 24454 24740 25932 26329 26846
 26891 26992 26993 27986 29177 29236 29239 31993 32095 47048 47093 47345
 47346 47363 47403 47531 47652 47697 47783 47788 47789 47790 47791 47792
 47796 47797 49164 81983 85932]
A0A2L0PNZ2_VITS1 0.09722222222222222 [ 5284  5303  5312  5370  5371  5378  5381  5383  6497  6498  6608  7135
  8224  8713  8927 10382 12784 13034 13035 13046 13058 13347 13972 15735
 17770 24006 24033 24096 26329 26846 26891 27495 27528 28387 29126 29298
 29323 29324 29336 29337 29339 30955 33721 46918 46964 46997 47341 47349
 47432 47491 47697 47788 47790 47796 47797 57230 91262]
A0A2L0PMR4_VITS1 0.06896551724137931 [ 5284  5370  5371  5378  5381  5383  6497  6498  6608  7135  8224  8927
 10382 12784 13034 13035 13046 13058 13347 13972 15735 17770 24006 24033
 24096 26329 26846 26891 27331 27986 29298 29303 31032 31034 31412 31420
 46784 46918 

AssertionError: 

In [58]:
0.15 + 0.09722 + 0.0689 + 0.107 + 0.107

0.53012

In [53]:
df.head()

,term,hierarchical_level,description,year,over_under,p_value,FDR,effectSize,ratio_in_FG,ratio_in_BG,FG_count,FG_n,BG_count,BG_n,FG_IDs,s_value,rank,funcEnum,category,etype
47,GO:0019538,5,protein metabolic process,-1,u,0.220052,1.0,-0.025,0.015,0.040,3,200,8,200,A0A2L0PJZ5_VITS1;A0A2L0PQL5_VITS1;A0A2L0PQM0_V...,-0.016437,1,8828,Gene Ontology biological process,-21
67,GO:0045333,6,cellular respiration,-1,o,0.215217,1.0,0.020,0.025,0.005,5,200,1,200,A0A2L0PMR4_VITS1;A0A2L0PNY0_VITS1;A0A2L0PNZ2_V...,0.013342,2,13347,Gene Ontology biological process,-21
9,GO:0006139,5,nucleobase-containing compound metabolic process,-1,o,0.382609,1.0,0.030,0.105,0.075,21,200,15,200,A0A2L0PID3_VITS1;A0A2L0PID8_VITS1;A0A2L0PIJ7_V...,0.012517,3,5407,Gene Ontology biological process,-21
6,GO:0006091,4,generation of precursor metabolites and energy,-1,o,0.284097,1.0,0.020,0.030,0.010,6,200,2,200,A0A2L0PMR4_VITS1;A0A2L0PNY0_VITS1;A0A2L0PNZ2_V...,0.010931,4,5378,Gene Ontology biological process,-21
74,GO:0055114,3,oxidation-reduction process,-1,o,0.284097,1.0,0.020,0.030,0.010,6,200,2,200,A0A2L0PMR4_VITS1;A0A2L0PNH0_VITS1;A0A2L0PNY0_V...,0.010931,5,15735,Gene Ontology biological process,-21


In [55]:
df.loc[67, "FG_IDs"]

'A0A2L0PMR4_VITS1;A0A2L0PNY0_VITS1;A0A2L0PNZ2_VITS1;A0A2L0PPD5_VITS1;Q8L311_VITS1'

In [46]:
ui.args_dict['ERROR foreground empty']

"Please check the foreground you've provided. It seems we can't find anything to parse."

In [38]:
ui.check_parse

True

In [37]:
ui.df_orig.head()

,background,intensity,foreground


In [79]:
df.head()

,term,hierarchical_level,description,year,over_under,p_value,FDR,effectSize,ratio_in_FG,ratio_in_BG,FG_count,FG_n,BG_count,BG_n,FG_IDs,s_value,rank,funcEnum,category,etype
1,GO:0016020,2,membrane,-1,u,9.992662e-07,0.002762,-0.122667,0.048000,0.170667,18,375,64,375,Q7AP69;Q8Y3N6;Q8Y3U8;Q8Y4C8;Q8Y527;Q8Y7P9;Q8Y8...,-0.736039,1,24453,Gene Ontology cellular component,-22
2,GO:0016021,5,integral component of membrane,-1,u,1.728198e-06,0.002762,-0.114667,0.040000,0.154667,15,375,58,375,Q8Y3N6;Q8Y3U8;Q8Y527;Q8Y7P9;Q8Y868;Q8Y8H7;Q8Y8...,-0.660756,2,24454,Gene Ontology cellular component,-22
3,GO:0071944,3,cell periphery,-1,u,1.476595e-05,0.010203,-0.074667,0.021333,0.096000,8,375,36,375,Q8Y3N6;Q8Y3U8;Q8Y527;Q8Y7P9;Q8Y833;Q8YAV5;P0DJ...,-0.360695,3,25932,Gene Ontology cellular component,-22
0,GO:0005886,4,plasma membrane,-1,u,1.531462e-05,0.010203,-0.072000,0.018667,0.090667,7,375,34,375,Q8Y3N6;Q8Y3U8;Q8Y527;Q8Y7P9;Q8Y833;Q8YAV5;P0DJP3,-0.346672,4,24207,Gene Ontology cellular component,-22
4,KW-0812,3,Transmembrane,-1,u,1.442517e-06,0.001694,-0.112000,0.034667,0.146667,13,375,55,375,Q8Y3N6;Q8Y3U8;Q8Y527;Q8Y7P9;Q8Y868;Q8Y8H7;Q8Y8...,-0.654178,1,47345,UniProt keywords,-51


In [68]:
print(ui.background["intensity"].min())
print(ui.foreground["intensity"].min())

7766643.143999999
7766643.143999999


In [ ]:
# foreground_ids_arr_of_string, background_ids_arr_of_string, funcEnum_count_foreground, funcEnum_count_background, p_values, p_values_corrected, cond_multitest, blacklisted_terms_bool_arr_temp, cond_terms_reduced_with_ontology, cond_filter, cond_PMIDs, effectSizes, over_under_int_arr, over_under_arr_of_string = preloaded_objects_per_analysis
len("""foreground_ids_arr_of_string, background_ids_arr_of_string, funcEnum_count_foreground, funcEnum_count_background, p_values, p_values_corrected, cond_multitest, blacklisted_terms_bool_arr_temp, cond_terms_reduced_with_ontology, cond_filter, cond_PMIDs, effectSizes, over_under_int_arr, over_under_arr_of_string""".split(", "))

In [20]:
preloaded_objects_per_analysis = pqo.get_preloaded_objects_per_analysis()
len(preloaded_objects_per_analysis)

14

In [22]:
reload(userinput)

<module 'userinput' from '/Users/dblyon/modules/cpr/agotool/app/python/userinput.py'>

In [43]:
ui.default_missing_bin

7766642.143999999

In [48]:
ui.background["intensity"].min(), ui.foreground["intensity"].min()

(7766643.143999999, 7766643.143999999)

In [69]:
col_intensity = "intensity"
DEFAULT_MISSING_BIN = -1
NUM_BINS = 100
# take subset of foreground with proper abundance values and create bins
cond = ui.foreground[col_intensity] > ui.default_missing_bin
bins = pd.cut(ui.foreground.loc[cond, col_intensity], bins=NUM_BINS, retbins=True)[1]
# add missing bin for the remainder of proteins
bins = np.insert(bins, 0, ui.default_missing_bin - 1)  # bins = [DEFAULT_MISSING_BIN - 1] + list(bins)
# cut foreground and background into bins
# groups_fg = ui.foreground.groupby(pd.cut(ui.foreground[col_intensity], bins=bins))

In [70]:
l1 = ui.foreground["intensity"].tolist()

In [71]:
l1 == sorted(l1)

True

In [72]:
bins = pd.cut(ui.foreground.loc[cond, col_intensity], bins=NUM_BINS, retbins=True)[1]
# bins = np.insert(bins, 0, ui.default_missing_bin - 1)  # bins = [DEFAULT_MISSING_BIN - 1] + list(bins)
l1 = list(bins)
l1 == sorted(l1)

True

In [75]:
-4.37220506e+07

-43722050.6

In [74]:
bins

array([-4.37220506e+07,  5.22653581e+08,  1.03754052e+09,  1.55242746e+09,
        2.06731439e+09,  2.58220133e+09,  3.09708827e+09,  3.61197521e+09,
        4.12686214e+09,  4.64174908e+09,  5.15663602e+09,  5.67152296e+09,
        6.18640990e+09,  6.70129683e+09,  7.21618377e+09,  7.73107071e+09,
        8.24595765e+09,  8.76084458e+09,  9.27573152e+09,  9.79061846e+09,
        1.03055054e+10,  1.08203923e+10,  1.13352793e+10,  1.18501662e+10,
        1.23650531e+10,  1.28799401e+10,  1.33948270e+10,  1.39097140e+10,
        1.44246009e+10,  1.49394878e+10,  1.54543748e+10,  1.59692617e+10,
        1.64841487e+10,  1.69990356e+10,  1.75139225e+10,  1.80288095e+10,
        1.85436964e+10,  1.90585833e+10,  1.95734703e+10,  2.00883572e+10,
        2.06032442e+10,  2.11181311e+10,  2.16330180e+10,  2.21479050e+10,
        2.26627919e+10,  2.31776788e+10,  2.36925658e+10,  2.42074527e+10,
        2.47223397e+10,  2.52372266e+10,  2.57521135e+10,  2.62670005e+10,
        2.67818874e+10,  

In [76]:
bins = pd.cut(ui.foreground.loc[cond, col_intensity], bins=NUM_BINS, retbins=True)[1]
bins = np.insert(bins, 0, ui.default_missing_bin - 1)  # bins = [DEFAULT_MISSING_BIN - 1] + list(bins)
bins

array([ 7.76664114e+06, -4.37220506e+07,  5.22653581e+08,  1.03754052e+09,
        1.55242746e+09,  2.06731439e+09,  2.58220133e+09,  3.09708827e+09,
        3.61197521e+09,  4.12686214e+09,  4.64174908e+09,  5.15663602e+09,
        5.67152296e+09,  6.18640990e+09,  6.70129683e+09,  7.21618377e+09,
        7.73107071e+09,  8.24595765e+09,  8.76084458e+09,  9.27573152e+09,
        9.79061846e+09,  1.03055054e+10,  1.08203923e+10,  1.13352793e+10,
        1.18501662e+10,  1.23650531e+10,  1.28799401e+10,  1.33948270e+10,
        1.39097140e+10,  1.44246009e+10,  1.49394878e+10,  1.54543748e+10,
        1.59692617e+10,  1.64841487e+10,  1.69990356e+10,  1.75139225e+10,
        1.80288095e+10,  1.85436964e+10,  1.90585833e+10,  1.95734703e+10,
        2.00883572e+10,  2.06032442e+10,  2.11181311e+10,  2.16330180e+10,
        2.21479050e+10,  2.26627919e+10,  2.31776788e+10,  2.36925658e+10,
        2.42074527e+10,  2.47223397e+10,  2.52372266e+10,  2.57521135e+10,
        2.62670005e+10,  

In [23]:
# ui.foreground.head()

In [24]:
# ui.df_orig.head()

In [16]:
len(static_preloaded_objects)

19

In [26]:
df.groupby("category")["term"].count()

category
Brenda Tissue Ontology                            3004
Disease Ontology                                  4609
Gene Ontology biological process                  2307
Gene Ontology cellular component                  2221
Gene Ontology molecular function                   329
INTERPRO                                           198
KEGG (Kyoto Encyclopedia of Genes and Genomes)      61
PFAM (Protein FAMilies)                             77
PMID (PubMed IDentifier)                           100
Reactome                                           229
UniProt keywords                                   159
WikiPathways                                        97
Name: term, dtype: int64

In [25]:
df.groupby("etype")["term"].count()

etype
-58      97
-57     229
-56     100
-55      77
-54     198
-52      61
-51     159
-26    4609
-25    3004
-23     329
-22    2221
-21    2307
Name: term, dtype: int64

In [22]:
df.groupby("etype").head(3)

,term,hierarchical_level,description,year,over_under,p_value,FDR,effectSize,ratio_in_FG,ratio_in_BG,FG_count,FG_n,BG_count,BG_n,FG_IDs,s_value,rank,funcEnum,category,etype
35,GO:0007264,7,small GTPase mediated signal transduction,-1,o,5.740206e-07,0.001620,0.284433,0.300000,0.015567,90,300,321,20621,RAB10_HUMAN;RAB12_HUMAN;RAB13_HUMAN;RAB14_HUMA...,1.775169,1,6181,Gene Ontology biological process,-21
36,GO:0007265,8,Ras protein signal transduction,-1,o,3.664623e-07,0.001620,0.274737,0.286667,0.011930,86,300,246,20621,RAB10_HUMAN;RAB12_HUMAN;RAB13_HUMAN;RAB14_HUMA...,1.768200,2,6182,Gene Ontology biological process,-21
69,GO:0035556,6,intracellular signal transduction,-1,o,1.913462e-06,0.001620,0.292154,0.373333,0.081179,112,300,1674,20621,RAB10_HUMAN;RAB12_HUMAN;RAB13_HUMAN;RAB14_HUMA...,1.670589,3,11464,Gene Ontology biological process,-21
266,GO:0005968,5,Rab-protein geranylgeranyltransferase complex,-1,o,7.765349e-07,0.000452,0.216251,0.236667,0.020416,71,300,421,20621,R7BP_HUMAN;RAB10_HUMAN;RAB13_HUMAN;RAB14_HUMAN...,1.321256,1,24383,Gene Ontology cellular component,-22
502,GO:0036284,3,tubulobulbar complex,-1,o,8.282181e-07,0.000452,0.205082,0.253333,0.048252,76,300,995,20621,R3GEF_HUMAN;RAB10_HUMAN;RAB12_HUMAN;RAB13_HUMA...,1.247276,2,25542,Gene Ontology cellular component,-22
605,GO:0055037,10,recycling endosome,-1,o,2.801950e-06,0.000452,0.214992,0.350000,0.135008,105,300,2784,20621,R3GEF_HUMAN;RAB10_HUMAN;RAB12_HUMAN;RAB13_HUMA...,1.193752,3,26100,Gene Ontology cellular component,-22
845,GO:0097159,3,organic cyclic compound binding,-1,o,3.038793e-06,0.001956,0.380380,0.673333,0.292954,202,300,6041,20621,QSOX1_HUMAN;R3HC1_HUMAN;R3HD1_HUMAN;R3HD2_HUMA...,2.098668,1,34160,Gene Ontology molecular function,-23
848,GO:1901363,3,heterocyclic compound binding,-1,o,3.737711e-06,0.001956,0.384744,0.673333,0.288589,202,300,5951,20621,QSOX1_HUMAN;R3HC1_HUMAN;R3HD1_HUMAN;R3HD2_HUMA...,2.088158,2,34673,Gene Ontology molecular function,-23
835,GO:0032550,6,purine ribonucleoside binding,-1,o,3.924690e-07,0.001956,0.284809,0.303333,0.018525,91,300,382,20621,RAB10_HUMAN;RAB12_HUMAN;RAB13_HUMAN;RAB14_HUMA...,1.824539,3,31219,Gene Ontology molecular function,-23
1230,BTO:0005590,4,Cervical adenocarcinoma cell line,-1,o,3.265207e-06,0.000672,0.215443,0.650000,0.434557,195,300,8961,20621,R113A_HUMAN;R144B_HUMAN;R3GEF_HUMAN;R3HCL_HUMA...,1.181940,1,38946,Brenda Tissue Ontology,-25


In [ ]:
# fn_userinput = os.path.join(variables.EXAMPLE_FOLDER, "Example_1_Yeast_acetylation_abundance_correction.txt")
# df = pd.read_csv(fn_userinput, sep='\t')
# fg = df.loc[df["Foreground"].notnull(), "Foreground"].tolist() # redundant UniProt accessions
# UniProtID_2_test_list = list(query.map_secondary_2_primary_ANs(fg).values()) # no more redundancy
# assert len(UniProtID_2_test_list) == 1159

# fg_scores_matrix, list_of_rowIndices_fg = slice_ScoresMatrix_for_given_ENSP(UniProtID_2_test_list, ENSP_2_rowIndex_dict, CSC_ENSPencoding_2_FuncEnum)

# funcEnum_2_scores_dict = {}
# missingScores_list = []
# for UniProtID in UniProtID_2_test_list:
#     try:
#         funcEnum_arr, score_arr = ENSP_2_tuple_funcEnum_score_dict[UniProtID]
#     except KeyError: #?
#         missingScores_list.append(UniProtID)
#         continue
#     for funcEnum, score in zip(funcEnum_arr, score_arr):
#         if funcEnum not in funcEnum_2_scores_dict:
#             funcEnum_2_scores_dict[funcEnum] = [score]
#         else:
#             funcEnum_2_scores_dict[funcEnum].append(score)

# m = fg_scores_matrix
# counter = 0
# for i in range(len(m.indptr[:-1])):  # get column values
#     index_row_start = m.indptr[i]
#     index_row_stop = m.indptr[i + 1]
#     if index_row_start == index_row_stop:
#         continue
#     funcEnum = i
#     scores_list_sparse = sorted(m.data[index_row_start:index_row_stop])
#     scores_list_ff = sorted(funcEnum_2_scores_dict[funcEnum])    
#     assert len(scores_list_sparse) == len(scores_list_ff)
#     assert scores_list_sparse == scores_list_ff
#     counter += 1
# print("# of tests {}".format(counter))

In [ ]:
funcEnum_2_scores_dict_bg[funcEnum]

In [ ]:
len(missingScores_list), missingScores_list

In [ ]:
scores_list_sparse, scores_list_ff

In [ ]:
fn_userinput = os.path.join(variables.EXAMPLE_FOLDER, "Example_1_Yeast_acetylation_abundance_correction.txt")
df = pd.read_csv(fn_userinput, sep='\t')
fg = df.loc[df["Foreground"].notnull(), "Foreground"].tolist()
d = list(query.map_secondary_2_primary_ANs(fg).values())

In [ ]:
uniprotid = 'YD21B_YEAST'
ENSP_2_tuple_funcEnum_score_dict["ARV1_ARATH"]

In [ ]:
dfs.to_csv("~/Downloads/Example_1_Yeast_acetylation_abundance_correction_OUTPUT_SCIPY.txt", sep="\t", header=True, index=False)
dfc.to_csv("~/Downloads/Example_1_Yeast_acetylation_abundance_correction_OUTPUT_CYTHON.txt", sep="\t", header=True, index=False)

In [ ]:
list(ENSP_2_tuple_funcEnum_score_dict.keys())[:3]

In [ ]:
print(len(fg), len(set(fg)))
fg[:3]

In [ ]:
# data = fg_scores_matrix_data
# indptr = fg_scores_matrix_indptr
# for i in range(len(indptr[:-1])):  # get column values
#     index_row_start = indptr[i]
#     index_row_stop = indptr[i + 1]
#     if index_row_start == index_row_stop:
#         continue
#     funcEnum = i
#     scores_list_sparse = sorted(data[index_row_start:index_row_stop])
#     scores_list_ff = sorted(funcEnum_2_scores_dict_fg[funcEnum])
#     assert len(scores_list_sparse) ==
#     len(scores_list_ff)
#     assert scores_list_sparse == scores_list_ff

In [ ]:
len(scores_list_sparse), len(scores_list_ff)

In [ ]:
funcEnum

In [ ]:
funcEnum = 256 # Nucleus
sum(funcEnum_2_scores_dict_fg[funcEnum] > 0)

In [ ]:
foreground_n, background_n

In [ ]:
# ui.foreground

In [ ]:
pqo.taxid_2_proteome_count[559292]

In [ ]:
print(dfc.groupby("etype")["term"].count())
print(dfs.groupby("etype")["term"].count())

In [ ]:
# dfc[dfc["etype"] == -20].head()

In [ ]:
dfs[dfs["term"] == "GOCC:0005634"].FG_IDs.values[0] == dfc[dfc["term"] == "GOCC:0005634"].FG_IDs.values[0]

In [ ]:
len(dfs[dfs["term"] == "GOCC:0005634"].FG_IDs.values[0].split(";")), len(dfc[dfc["term"] == "GOCC:0005634"].FG_IDs.values[0].split(";"))

In [ ]:
dfs[dfs["term"] == "GOCC:0005634"]

In [ ]:
dfc[dfc["term"] == "GOCC:0005634"]

In [ ]:
0.986195 - 0.950679

In [ ]:
dfs.groupby("etype").term.count(), dfc.groupby("etype").term.count()

In [ ]:
126/177

In [ ]:
# dfs[dfs["etype"] == -20]

In [ ]:
# dfc = df.copy()
# dfc.groupby("etype").head(2)

In [ ]:
# dfs = df.copy()
# dfs.groupby("etype").head(2)

In [ ]:
dfs[dfs["etype"] == -20].head()

In [ ]:
dfc[dfc["etype"] == -20].head()

In [ ]:
# Yeast acetylation, underrepresented, scipy
print(df.shape)
print(df.groupby("etype")["term"].count())

In [ ]:
# Yeast acetylation, underrepresented, cython
print(df.shape)
print(df.groupby("etype")["term"].count())

In [ ]:
# Yeast acetylation, both, scipy
print(df.shape)
print(df.groupby("etype")["term"].count())

In [ ]:
# Yeast acetylation, both, cython
print(df.shape)
print(df.groupby("etype")["term"].count())

In [ ]:
# Yeast acetylation, overrepresented, cython
print(df.shape)
print(df.groupby("etype")["term"].count())

In [ ]:
df[df["etype"] == -51]

In [ ]:
df.groupby("etype").head(2)

In [ ]:
# Problematic call
foreground_input = '511145.b1260%0d511145.b1261%0d511145.b1262%0d511145.b1263%0d511145.b1264%0d511145.b1812%0d511145.b2551%0d511145.b3117%0d511145.b3772%0d511145.b1015%0d511145.b2585'
foreground_input = foreground_input.split("%0d")
background_input = []

enrichment_method = "genome"
args_dict = {}
args_dict["enrichment_method"] = enrichment_method
args_dict["taxid"] = 511145
args_dict["FDR_cutoff"] = 0.05
args_dict["p_value_cutoff"] = 0.01
args_dict["limit_2_entity_type"] = None # "-20;-25;-26" #"-21;-22;-23;-51;-52;-53;-54;-55;-56-57;-58"
args_dict["filter_PMID_top_n"] = 100
args_dict["filter_foreground_count_one"] = False
args_dict["filter_parents"] = False
args_dict["go_slim_subset"] = None # "generic"
args_dict["o_or_u_or_both"] = "both" # "both" "underrepresented" "overrepresented"
args_dict["multiple_testing_per_etype"] = True
args_dict["privileged"] = True
args_dict["score_cutoff"] = 0
args_dict["caller_identity"] = "11_0"
taxid = args_dict["taxid"]
debug = False
profile = False
simplified_output = False
args_dict["simplified_output"] = simplified_output
args_dict["STRING_beta"] = True
KS_method = "cy"
# result = requests.post(url_, params={"output_format": "tsv", "enrichment_method": "genome", "taxid": 511145, "caller_identity": "11_0", "STRING_beta": True, 
#                                      'FDR_cutoff': '0.05'}, data={"foreground": fg, "background": bg})
ui = userinput.Userinput(pqo, fn=None, foreground_string=userinput.stringify_for_Userinput(foreground_input), background_string=userinput.stringify_for_Userinput(background_input), args_dict=args_dict)
preloaded_objects_per_analysis = pqo.get_preloaded_objects_per_analysis()
if debug:
    funcEnum_2_scores_dict_bg, foreground_n, background_n, fg_scores_matrix_data, fg_scores_matrix_indptr, bg_scores_matrix_data, bg_scores_matrix_indptr, p_values, cond_multitest, effectSizes, p_value_cutoff, funcEnum_count_foreground, funcEnum_count_background, over_under_int_arr, o_or_u_or_both_encoding, em, filter_foreground_count_one = run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug, KS_method=KS_method, ENSP_2_tuple_funcEnum_score_dict=ENSP_2_tuple_funcEnum_score_dict)
else:
    df = run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True, debug=debug, KS_method=KS_method, ENSP_2_tuple_funcEnum_score_dict=ENSP_2_tuple_funcEnum_score_dict)

In [ ]:
df.groupby("etype").head(2)

In [ ]:
# import requests
# from io import StringIO
# import pandas as pd
# # call api_help for help and argument defaults
# response = requests.get(r"https://agotool.org/api_help")
# print(response.json())
# url_ = r"https://agotool.org/api"
# # url_ = r"http://localhost:5000/api"

In [ ]:
# ENSPs = ['4932.YAR019C', '4932.YFR028C', '4932.YGR092W', '4932.YHR152W', '4932.YIL106W', '4932.YJL076W',
#      '4932.YLR079W', '4932.YML064C', '4932.YMR055C', '4932.YOR373W', '4932.YPR119W']
# fg = "%0d".join(ENSPs)
# result = requests.post(url_,
#                    params={"output_format": "tsv",
#                            "enrichment_method": "genome",
#                            "taxid": 559292, "STRING_beta": True}, 
#                        # UniProt reference proteomes uses "Saccharomyces cerevisiae S288C" with Taxid 559292 as a pan proteome instead of 4932 (TaxID on taxonomic rank of species).
#                    data={"foreground": fg})
# result = result.text
# df = pd.read_csv(StringIO(result), sep='\t')
# df.head(2)

In [ ]:
# import requests
# from io import StringIO
# import pandas as pd

# url_ = r"https://agotool.org/api"
# ENSPs = ['4932.YAR019C', '4932.YFR028C', '4932.YGR092W', '4932.YHR152W', '4932.YIL106W', '4932.YJL076W',
#      '4932.YLR079W', '4932.YML064C', '4932.YMR055C', '4932.YOR373W', '4932.YPR119W']
# fg = "%0d".join(ENSPs)
# result = requests.post(url_,
#                    params={"output_format": "tsv",
#                            "enrichment_method": "genome",
#                            "taxid": 559292, "STRING_beta": False}, 
#                        # UniProt reference proteomes uses "Saccharomyces cerevisiae S288C" with Taxid 559292 as a pan proteome instead of 4932 (TaxID on taxonomic rank of species).
#                    data={"foreground": fg})
# result = result.text
# df = pd.read_csv(StringIO(result), sep='\t')
# df.head(2)

In [ ]:
# ToDo PyTests 2 write
# compare ENSP_2_tuple_funcEnum_score_dict with Sparse Matrix (funcEnum_2_scores_dict_fg, funcEnum_2_scores_dict_bg). 
#   For any given input (of taxid 9606) check that FG_count <= BG_count
#   funcEnums of FG must also be in BG
#   scores of FG must also be in BG

# for given Protein what functions are associated?

In [ ]:
fg_scores_matrix, list_of_rowIndices_fg = slice_ScoresMatrix_for_given_ENSP(protein_ans_fg, ENSP_2_rowIndex_dict, CSC_ENSPencoding_2_FuncEnum)
fg_scores_matrix_data = fg_scores_matrix.data
fg_scores_matrix_indptr = fg_scores_matrix.indptr
set_fg_counts(fg_scores_matrix_data, fg_scores_matrix_indptr, funcEnum_count_foreground, filter_foreground_count_one)
# add_funcEnums_2_dict(protein_ans_fg, ENSP_2_functionEnumArray_dict, ENSP_2_tuple_funcEnum_score_dict)
add_funcEnums_2_dict_CSC(protein_ans_fg, ENSP_2_functionEnumArray_dict, ENSP_2_rowIndex_dict, CSR_ENSPencoding_2_FuncEnum)

In [ ]:
# How can the FG_count always be as high as FG_n?
# Why is BG_count not >= as FG_count? --> PyTest
funcEnum_2_scores_dict_bg = Taxid_2_FunctionEnum_2_Scores_dict[9606]

# The plan

In [ ]:
# e.g. GOCC:0043226; 1893 (funcEnum) with 59 BG_count, but 97 FG_count and enrichment_method "genome"
# --> PYTEST: FG_count should always be <= BG_count
# CHECK:
# Which files are in the pipeline? 
    # - Lars download with ENSPs
    # - translated to UniProtIDs, scaled values from float to int and backtracked
    # - counted per TaxID
    
# Which ENSPs are associated with GOCC:0043226 in original Lars download? 
# Which UniProtIDs are associated with GOCC:0043226 in original Lars download?
# How many and which

In [ ]:
fg_scores_matrix, list_of_rowIndices_fg = slice_ScoresMatrix_for_given_ENSP(protein_ans_fg, ENSP_2_rowIndex_dict, CSC_ENSPencoding_2_FuncEnum)
fg_scores_matrix_data = fg_scores_matrix.data
fg_scores_matrix_indptr = fg_scores_matrix.indptr

In [ ]:
ui.df_orig.shape

In [ ]:
collect_scores_per_term_limit_2_inclusionTerms(protein_ans_fg, ENSP_2_tuple_funcEnum_score_dict, funcEnums_2_include_set, list_2_array=True)

In [ ]:
prot_noFunc_list, scores_list = [], []
funcEnum = 1893
protein_ans_fg = ui.get_all_individual_foreground_ANs()
for prot in protein_ans_fg:
    try:
        funcEnum_arr, score_arr = ENSP_2_tuple_funcEnum_score_dict[prot]
    except:
        prot_noFunc_list.append(prot)
        continue
    x = np.where(funcEnum_arr == funcEnum)[0]
    if x.shape[0] == 0:
        print("")
        continue
    else:
        index_ = x[0]
    scores_list.append(score_arr[index_])

In [ ]:
len(scores_list), len(prot_noFunc_list)

In [ ]:
funcEnum_arr, score_arr = ENSP_2_tuple_funcEnum_score_dict["GPT_HUMAN"]
np.where(funcEnum_arr == funcEnum)
# index_ = np.where(funcEnum_arr == funcEnum)[0][0]
# print(score_arr[index_])

# Profiling

## cProfile

In [ ]:
del run_cythonized

In [ ]:
import run_cythonized
reload(run_cythonized)

In [ ]:
# import cProfile
# cProfile.run('run_cythonized.run_genome_cy(taxid, protein_ans, background_n, preloaded_objects_per_analysis, static_preloaded_objects, args_dict, low_memory=False)', sort='time') > prof_temp.txt

## line profiler

In [ ]:
## https://stackoverflow.com/questions/28301931/how-to-profile-cython-functions-line-by-line
## add this to Jupyter Notebook
#%%cython -f --compile-args=-DCYTHON_TRACE=1
#import Cython
######################################
### profiling # Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
import line_profiler
directive_defaults = Cython.Compiler.Options.get_directive_defaults() ### from Cython.Compiler.Options import directive_defaults # deprecated
directive_defaults['linetrace'] = True
directive_defaults['binding'] = True
######################################

## then run this to profile
# %load_ext line_profiler
import line_profiler

fn_userinput = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/ExampleData_for_testing.txt"
foreground_input = ["HBA_HUMAN", "HBB_HUMAN", "HBD_HUMAN", "HBE_HUMAN"]
taxid = 9606
from_file = False

args_dict = {}
args_dict["enrichment_method"] = "genome"
args_dict["FDR_cutoff"] = 0.05
args_dict["p_value_cutoff"] = 0.01
args_dict["limit_2_entity_type"] = "-20;-25;-21" # "-20;-21;-22;-23;-25;-26" # None #"-21;-22;-23"
args_dict["filter_PMID_top_n"] = 50
args_dict["filter_foreground_count_one"] = True
args_dict["filter_parents"] = True
args_dict["go_slim_subset"] = None # "generic"
args_dict["taxid"] = taxid
args_dict["o_or_u_or_both"] = "both" # "both" "underrepresented"
args_dict["multiple_testing_per_etype"] = True
args_dict["privileged"] = True

background_input = query.get_proteins_of_taxid(taxid, read_from_flat_files=True)
if from_file:
    ui = userinput.Userinput(pqo, fn_userinput, args_dict=args_dict)
else:
    ui = userinput.Userinput(pqo, fn=None, foreground_string=userinput.stringify_for_Userinput(foreground_input), background_string=userinput.stringify_for_Userinput(background_input), args_dict=args_dict)
ncbi = pqo.ncbi

preloaded_objects_per_analysis = pqo.get_preloaded_objects_per_analysis()
profile = line_profiler.LineProfiler(run_enrichment_cy)
profile.runcall(run_enrichment_cy, ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True)
profile.print_stats()
# > 70% of time spent on KS (Hemoglobin foreground, genome background). 76.7 (only categories with scores), 69.8 (all categories) 
#  --> test KS funtion alone

In [ ]:
### get objects for code block below
foreground_input = ["HBA_HUMAN", "HBB_HUMAN", "HBD_HUMAN", "HBE_HUMAN"] # example 1: Human hemoglobin, genome
enrichment_method = "genome" # "characterize_foreground" "abundance_correction" "compare_samples" "genome" "compare_groups"
from_file = False
args_dict = {}
args_dict["enrichment_method"] = enrichment_method
args_dict["taxid"] = 9606
args_dict["FDR_cutoff"] = 0.05
args_dict["p_value_cutoff"] = 0.01
args_dict["limit_2_entity_type"] = "-20;-25;-26" #"-20;-25;-21" # "-20;-21;-22;-23;-25;-26" # None #"-21;-22;-23"
args_dict["filter_PMID_top_n"] = 50
args_dict["filter_foreground_count_one"] = True
args_dict["filter_parents"] = True
args_dict["go_slim_subset"] = None # "generic"
args_dict["o_or_u_or_both"] = "overrepresented" # "both" "underrepresented"
args_dict["multiple_testing_per_etype"] = True
args_dict["privileged"] = True
args_dict["score_cutoff"] = 0
args_dict["foreground_replicates"] = 10
args_dict["background_replicates"] = 10
taxid = args_dict["taxid"]
background_input = query.get_proteins_of_taxid(taxid, read_from_flat_files=True)
if from_file:
    ui = userinput.Userinput(pqo, fn_userinput, args_dict=args_dict)
else:
    ui = userinput.Userinput(pqo, fn=None, foreground_string=userinput.stringify_for_Userinput(foreground_input), background_string=userinput.stringify_for_Userinput(background_input), args_dict=args_dict)
ncbi = pqo.ncbi
preloaded_objects_per_analysis = pqo.get_preloaded_objects_per_analysis()
foreground_n, background_n, funcEnum_2_scores_dict_fg, funcEnum_2_scores_dict_bg, p_values, cond_multitest, effectSizes, p_value_cutoff, funcEnum_count_foreground, funcEnum_count_background, over_under_int_arr, o_or_u_or_both_encoding = run_enrichment_cy(ncbi, ui, preloaded_objects_per_analysis, static_preloaded_objects, low_memory=True)

In [ ]:
from scipy.stats import distributions
def ks_2samp_dbl(data1, data2):
    data1 = np.sort(data1)
    data2 = np.sort(data2)
    data1 = np.array(data1)
    data2 = np.array(data2)
    n1 = data1.shape[0]
    n2 = data2.shape[0]
    data_all = np.concatenate([data1, data2])
    cdf1 = np.searchsorted(data1, data_all, side='right') / n1
    cdf2 = np.searchsorted(data2, data_all, side='right') / n2
    d = np.max(np.absolute(cdf1 - cdf2))
    # Note: d absolute not signed distance
    en = np.sqrt(n1 * n2 / (n1 + n2))
    try:
        prob = distributions.kstwobign.sf((en + 0.12 + 0.11 / en) * d)
    # except Exception:
        # warnings.warn('This should not happen! Please open an issue at '
        #             'https://github.com/scipy/scipy/issues and provide the code '
        #            'you used to trigger this warning.\n')
        # prob = 1.0
    except:
        print("This shouldn't happen")
        raise StopIteration

    # return Ks_2sampResult(d, prob)
    return d, prob

In [ ]:
### fill with zeros for proper calculation and performance evaluation
funcEnum = 68613
scores_fg = funcEnum_2_scores_dict_fg[funcEnum]
scores_bg = funcEnum_2_scores_dict_bg[funcEnum]
scores_bg = list(scores_bg) # for genome method --> not the place to do this        
len_scores_fg = len(scores_fg)
number_of_zeros_2_fill = foreground_n - len_scores_fg
if number_of_zeros_2_fill > 0:
    scores_fg = [0]*number_of_zeros_2_fill + scores_fg
len_scores_bg = len(scores_bg)
number_of_zeros_2_fill = background_n - len_scores_bg
if number_of_zeros_2_fill > 0:
    scores_bg = [0]*number_of_zeros_2_fill + scores_bg    
data1, data2 = scores_fg, scores_bg

d_dbl, prob_dbl = ks_2samp_dbl(data1, data2)
d, prob = stats.ks_2samp(data1, data2, alternative="two-sided", mode="asymp")
print(d_dbl, prob_dbl)
assert d_dbl == d
# assert prob_dbl == prob
prob_dbl == prob

In [ ]:
def list_of_floats_2_R_style(vals, max_num_vals_per_line=100):
    vals = [str(ele) for ele in vals]
    string_2_return = "<-c("
    while len(vals) > 0:
        string_2_return += "{},\n".format(", ".join(vals[:max_num_vals_per_line]))
        vals = vals[max_num_vals_per_line:]
    return string_2_return[:-2] + ")"

In [ ]:
def transformation_to_randomize_sort_order(fg, bg):
    # value = value * ((double) 1.0 + ((double) rand()) / (((double) RAND_MAX) * (double) 1000.0)); // transformation of fold_change values in order to randomize sort order in case of ties
    np.random.seed(12345)
#     fg = [value * (1.0 + np.random.randint(0, 2147483647) / (2147483647.0 * 1000.0)) for value in fg]
    # bg = [value * (1.0 + np.random.randint(0, 2147483647) / (2147483647.0 * 1000.0)) for value in bg]
    fg = [value + np.random.randint(0, 2147483647) / (2147483647.0 * 1000.0) for value in fg]
    bg = [value + np.random.randint(0, 2147483647) / (2147483647.0 * 1000.0) for value in bg]
    return fg, bg

In [ ]:
funcEnum = 68613
scores_fg = funcEnum_2_scores_dict_fg[funcEnum]
scores_bg = funcEnum_2_scores_dict_bg[funcEnum]
print(stats.ks_2samp(scores_fg, scores_bg, alternative="two-sided", mode="exact"))
scores_fg, scores_bg = transformation_to_randomize_sort_order(scores_fg, scores_bg)
print(stats.ks_2samp(scores_fg, scores_bg, alternative="two-sided", mode="exact"))
print(stats.ks_2samp(data1, data2, alternative="two-sided", mode="exact"))
data1, data2 = transformation_to_randomize_sort_order(data1, data2)
print(stats.ks_2samp(data1, data2, alternative="two-sided", mode="exact"))

In [ ]:
%matplotlib inline
dfx = pd.DataFrame()
dfx["bg"] = pd.Series(scores_bg)
dfx["bg"].hist(cumulative=True, density=1, bins=100, alpha=0.2)
dfx["fg"] = pd.Series(scores_fg)
dfx["fg"].hist(cumulative=True, density=1, bins=100, alpha=0.2)

In [ ]:
def fg_and_bg_2_DF_melt_and_add_ranks(fg, bg):
    dfx = pd.DataFrame()
    dfx["bg"] = pd.Series(bg)
    dfx["fg"] = pd.Series(fg)
    # transform dataframe to long format
    dfxm = dfx.melt(value_vars=["fg", "bg"], var_name="FG_BG", value_name="score")
    dfxm = dfxm[dfxm["score"].notnull()]
    dfxm = dfxm.sort_values(["score", "FG_BG"], ascending=[True, False]).reset_index(drop=True)
    dfxm["rank"] = dfxm["score"].rank(method="first")
    return dfxm
# scores_fg = funcEnum_2_scores_dict_fg[funcEnum]
# scores_bg = funcEnum_2_scores_dict_bg[funcEnum]
# dfxm = fg_and_bg_2_DF_melt_and_add_ranks(scores_fg, scores_bg)
# print(stats.ks_2samp(scores_fg, scores_bg, alternative="two-sided", mode="exact"))
# stats.ks_2samp(dfxm.loc[dfxm["FG_BG"] == "fg", "score"].to_list(), dfxm.loc[dfxm["FG_BG"] == "bg", "score"].to_list(), alternative="two-sided", mode="exact")

In [ ]:
# scores_fg = funcEnum_2_scores_dict_fg[funcEnum]
# sorted(scores_fg)

In [ ]:
### Print profiling statistics using the `line_profiler` API
### 'run_cythonized.run_genome_cy(taxid, protein_ans, background_n, preloaded_objects_per_analysis, static_preloaded_objects, args_dict, low_memory=False)'
profile = line_profiler.LineProfiler(run_genome_cy)
profile.runcall(run_genome_cy, taxid, protein_ans, background_n, preloaded_objects_per_analysis, static_preloaded_objects, args_dict, low_memory=False)
profile.print_stats()